In [1]:
require(data.table)
install.packages("manipulateWidget")
require(manipulateWidget)
install.packages("dtw")
require(dtw)
require(rattle)
require(repr)
require(TSrepr)
install.packages("crosstalk")
require(crosstalk)
install.packages("TSdist")
require(TSdist)
install.packages("TunePareto")
require(TunePareto)
require(zoo)

Loading required package: data.table
Warning message:
"package 'data.table' was built under R version 3.6.3"


  There is a binary version available but the source version is later:
                 binary source needs_compilation
manipulateWidget 0.10.1 0.11.1             FALSE



installing the source package 'manipulateWidget'

Loading required package: manipulateWidget


package 'dtw' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\mirac\AppData\Local\Temp\RtmpEPNrup\downloaded_packages


Loading required package: dtw
Warning message:
"package 'dtw' was built under R version 3.6.3"Loading required package: proxy
Warning message:
"package 'proxy' was built under R version 3.6.3"
Attaching package: 'proxy'

The following objects are masked from 'package:stats':

    as.dist, dist

The following object is masked from 'package:base':

    as.matrix

Loaded dtw v1.22-3. See ?dtw for help, citation("dtw") for use in publication.

Loading required package: rattle
Warning message:
"package 'rattle' was built under R version 3.6.3"Loading required package: tibble
Loading required package: bitops
Warning message:
"package 'bitops' was built under R version 3.6.3"Rattle: A free graphical interface for data science with R.
Version 5.4.0 Copyright (c) 2006-2020 Togaware Pty Ltd.
Type 'rattle()' to shake, rattle, and roll your data.
Loading required package: repr
Loading required package: TSrepr
Warning message:
"package 'TSrepr' was built under R version 3.6.3"

...

installing the source package 'crosstalk'

Loading required package: crosstalk


package 'TSdist' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\mirac\AppData\Local\Temp\RtmpEPNrup\downloaded_packages


Loading required package: TSdist
Warning message:
"package 'TSdist' was built under R version 3.6.3"Registered S3 method overwritten by 'xts':
  method     from
  as.zoo.xts zoo 
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 
Loaded TSdist v3.7. See ?TSdist for help, citation("TSdist") for use in publication.




  There is a binary version available but the source version is later:
           binary source needs_compilation
TunePareto    2.5  2.5.1              TRUE

  Binaries will be installed
package 'TunePareto' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\mirac\AppData\Local\Temp\RtmpEPNrup\downloaded_packages


Loading required package: TunePareto
Warning message:
"package 'TunePareto' was built under R version 3.6.3"Loading required package: zoo

Attaching package: 'zoo'

The following objects are masked from 'package:base':

    as.Date, as.Date.numeric



### Reading Data and Manipulation

In [2]:
traindata_trace=fread("C:/Users/mirac/HW3/Trace/Trace_TRAIN.txt")
 setnames(traindata_trace,'V1','class')
 traindata_trace=traindata_trace[order(class)]
 traindata_trace[,class:=as.character(class)]
 traindata_trace[,id:=1:.N]
 longtrain_trace=melt(traindata_trace,id.vars=c('id','class'))
longtrain_trace[,time:=as.numeric(gsub("\\D", "", variable))-1]
longtrain_trace=longtrain_trace[,list(id,class,time,value)]
longtrain_trace=longtrain_trace[order(id,time)]


### Alternative Representations  
#### Piecewise Aggregate Approximation with segment length 4 and 5  
#### Piecewise Linear Approximation with important points 5 and 10 

In [3]:
reprfunc=function(x){
    selected_series=x
    data_plot=longtrain_trace[id==x]
    
    
    segment_length=5
    longtrain_trace=longtrain_trace[order(id,time)]
    data_ts=longtrain_trace[id==x]$value
    paa_rep5=repr_paa(data_ts, segment_length, meanC)
    
    dummy_time=c(1:(length(paa_rep5)-1))*segment_length 
    dummy_time=c(dummy_time,nrow(data_plot)) 
    
    dt_paa=data.table(time=dummy_time,paa_rep5=paa_rep5)
    data_plot=merge(data_plot,dt_paa,by='time',all.x=T)
    data_plot[,paa_rep5:=nafill(paa_rep5,'nocb')]
    
    
    segment_length=4
    longtrain_trace=longtrain_trace[order(id,time)]
    data_ts=longtrain_trace[id==x]$value
    paa_rep4=repr_paa(data_ts, segment_length, meanC)
    
    dummy_time=c(1:(length(paa_rep4)-1))*segment_length 
    dummy_time=c(dummy_time,nrow(data_plot)) 
    
    dt_paa=data.table(time=dummy_time,paa_rep4=paa_rep4)
    data_plot=merge(data_plot,dt_paa,by='time',all.x=T)
    data_plot[,paa_rep4:=nafill(paa_rep4,'nocb')]
    
    
    n_important = 5
    pla_rep5=repr_pla(data_ts, times = n_important, return = "both")
    setnames(pla_rep5,c('places','points'),c('time','pla_rep5'))
    pla_rep5$time=pla_rep5$time+1
    
    data_plot=merge(data_plot,pla_rep5,by='time',all.x=T)
    data_plot[, pla_rep5 := na.approx(pla_rep5)]
    
    
    n_important = 10
    pla_rep10=repr_pla(data_ts, times = n_important, return = "both")
    setnames(pla_rep10,c('places','points'),c('time','pla_rep10'))
    pla_rep10$time=pla_rep10$time+1
    
    data_plot=merge(data_plot,pla_rep10,by='time',all.x=T)
    data_plot[, pla_rep10 := na.approx(pla_rep10)]
    return(data_plot)
}


representations=data.table()

for(i in 1:100){
    representations=rbind(representations,reprfunc(i))
}


paa_rep5=dcast(representations,id~time,value.var='paa_rep5')
paa_rep4=dcast(representations,id~time,value.var='paa_rep4')
pla_rep5=dcast(representations,id~time,value.var='pla_rep5')
pla_rep10=dcast(representations,id~time,value.var='pla_rep10')

### Storing Distance Measures for Raw Data
#### For Euclidean / Dtw / Lcss with epsilon 0.1 / Erp with g 0.5 
#### With NN parameter k= 1,3,5

In [4]:
trainclass_trace=as.matrix(traindata_trace[,1])
large_number=10000
dist_euc=as.matrix(dist(traindata_trace[,2:ncol(traindata_trace)]))
diag(dist_euc)=large_number
fwrite(dist_euc,sprintf('%sTRACE_euc_raw_dist.csv',"C:/Users/mirac/HW3/Trace/Distances/Raw/"),col.names=F)
dist_dtw=as.matrix(dtwDist(traindata_trace[,2:ncol(traindata_trace)]))
diag(dist_dtw)=large_number
fwrite(dist_dtw,sprintf('%sTRACE_dtw_raw_dist.csv',"C:/Users/mirac/HW3/Trace/Distances/Raw/"),col.names=F)
dist_lcss=as.matrix(TSDatabaseDistances(as.matrix(traindata_trace[,2:ncol(traindata_trace)]),distance='lcss',epsilon=0.1))
diag(dist_lcss)=large_number
fwrite(dist_lcss,sprintf('%sTRACE_lcss_raw_epsilon_005.csv',"C:/Users/mirac/HW3/Trace/Distances/Raw/"),col.names=F)
dist_erp=as.matrix(TSDatabaseDistances(as.matrix(traindata_trace[,2:ncol(traindata_trace)]),distance='erp',g=0.5))
diag(dist_erp)=large_number
fwrite(dist_erp,sprintf('%sTRACE_erp_raw_gap_005.csv',"C:/Users/mirac/HW3/Trace/Distances/Raw/"),col.names=F)


x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table


### Function to Evaluate Cross Validation

In [5]:
nn_classify_cv=function(dist_matrix,train_class,test_indices,k=1){
    
    test_distances_to_train=dist_matrix[test_indices,]
    test_distances_to_train=test_distances_to_train[,-test_indices]
    train_class=train_class[-test_indices]
    ordered_indices=apply(test_distances_to_train,1,order)
    if(k==1){
        nearest_class=as.numeric(trainclass_trace[as.numeric(ordered_indices[1,])])
        nearest_class=data.table(id=test_indices,nearest_class)
    } else {
        nearest_class=apply(ordered_indices[1:k,],2,function(x) {trainclass_trace[x]})
        nearest_class=data.table(id=test_indices,t(nearest_class))
    }
    
    long_nn_class=melt(nearest_class,'id')
    
    class_counts=long_nn_class[,.N,list(id,value)]
    class_counts[,predicted_prob:=N/k]
    wide_class_prob_predictions=dcast(class_counts,id~value,value.var='predicted_prob')
    wide_class_prob_predictions[is.na(wide_class_prob_predictions)]=0
    class_predictions=class_counts[,list(predicted=value[which.max(N)]),by=list(id)]
    
    
    return(list(prediction=class_predictions,prob_estimates=wide_class_prob_predictions))
    
}

### Evalution of Raw Data

In [6]:
set.seed(482)
nof_rep=5
n_fold=10
cv_indices=generateCVRuns(trainclass_trace, ntimes =nof_rep, nfold = n_fold, leaveOneOut = FALSE, stratified = TRUE)
dist_files=list.files("C:/Users/mirac/HW3/Trace/Distances/Raw/", full.names=T)
k_levels=c(1,3,5)
approach_file=list.files("C:/Users/mirac/HW3/Trace/Distances/Raw/")
result=vector('list',length(dist_files)*nof_rep*n_fold*length(k_levels))
iter=1
for(m in 1:length(dist_files)){ 

    dist_mat=as.matrix(fread(dist_files[m],header=FALSE))
    for(i in 1:nof_rep){
        this_fold=cv_indices[[i]]
        for(j in 1:n_fold){
            test_indices=this_fold[[j]]
            for(k in 1:length(k_levels)){
                current_k=k_levels[k]
                current_fold=nn_classify_cv(dist_mat,trainclass_trace,test_indices,k=current_k)
                accuracy=sum(trainclass_trace[test_indices]==current_fold$prediction$predicted)/length(test_indices)
                tmp=data.table(approach=approach_file[m],repid=i,foldid=j,
                               k=current_k,acc=accuracy)
                result[[iter]]=tmp
                iter=iter+1
                
            }
            
        }
        
    }   
    
}
overall_results=rbindlist(result)
summarized_results=overall_results[,list(mean_acc=mean(acc),sdev_acc=sd(acc),result_count=.N),by=list(approach,k)]
summarized_results[order(-mean_acc)]


approach,k,mean_acc,sdev_acc,result_count
TRACE_erp_raw_gap_005.csv,5,0.920,0.08329931,50
TRACE_erp_raw_gap_005.csv,3,0.886,0.09037338,50
TRACE_dtw_raw_dist.csv,5,0.882,0.09623526,50
TRACE_erp_raw_gap_005.csv,1,0.870,0.10546302,50
TRACE_dtw_raw_dist.csv,3,0.838,0.10079278,50
TRACE_euc_raw_dist.csv,1,0.838,0.10279760,50
TRACE_euc_raw_dist.csv,3,0.834,0.10022424,50
TRACE_dtw_raw_dist.csv,1,0.832,0.09781282,50
TRACE_euc_raw_dist.csv,5,0.832,0.10961901,50
TRACE_lcss_raw_epsilon_005.csv,1,0.308,0.09441550,50


### Evaluating Piecewise Aggregate Approximation with segment length 4 and 5

In [7]:
dist_euc=as.matrix(dist(paa_rep4[,2:ncol(paa_rep4)]))
diag(dist_euc)=large_number
fwrite(dist_euc,sprintf('%sTRACE_euc_paa4_dist.csv',"C:/Users/mirac/HW3/Trace/Distances/Paa4/"),col.names=F)
dist_dtw=as.matrix(dtwDist(paa_rep4[,2:ncol(paa_rep4)]))
diag(dist_dtw)=large_number
fwrite(dist_dtw,sprintf('%sTRACE_dtw_paa4_dist.csv',"C:/Users/mirac/HW3/Trace/Distances/Paa4/"),col.names=F)
dist_lcss=as.matrix(TSDatabaseDistances(as.matrix(paa_rep4[,2:ncol(paa_rep4)]),distance='lcss',epsilon=0.1))
diag(dist_lcss)=large_number
fwrite(dist_lcss,sprintf('%sTRACE_lcss_paa4_epsilon_005.csv',"C:/Users/mirac/HW3/Trace/Distances/Paa4/"),col.names=F)
dist_erp=as.matrix(TSDatabaseDistances(as.matrix(paa_rep4[,2:ncol(paa_rep4)]),distance='erp',g=0.5))
diag(dist_erp)=large_number
fwrite(dist_erp,sprintf('%sTRACE_erp_paa4_gap_005.csv',"C:/Users/mirac/HW3/Trace/Distances/Paa4/"),col.names=F)

set.seed(482)
nof_rep=5
n_fold=10
cv_indices=generateCVRuns(trainclass_trace, ntimes =nof_rep, nfold = n_fold, leaveOneOut = FALSE, stratified = TRUE)
dist_files=list.files("C:/Users/mirac/HW3/Trace/Distances/Paa4/", full.names=T)
k_levels=c(1,3,5)
approach_file=list.files("C:/Users/mirac/HW3/Trace/Distances/Paa4/")
result=vector('list',length(dist_files)*nof_rep*n_fold*length(k_levels))
iter=1
for(m in 1:length(dist_files)){ 

    dist_mat=as.matrix(fread(dist_files[m],header=FALSE))
    for(i in 1:nof_rep){
        this_fold=cv_indices[[i]]
        for(j in 1:n_fold){
            test_indices=this_fold[[j]]
            for(k in 1:length(k_levels)){
                current_k=k_levels[k]
                current_fold=nn_classify_cv(dist_mat,trainclass_trace,test_indices,k=current_k)
                accuracy=sum(trainclass_trace[test_indices]==current_fold$prediction$predicted)/length(test_indices)
                tmp=data.table(approach=approach_file[m],repid=i,foldid=j,
                               k=current_k,acc=accuracy)
                result[[iter]]=tmp
                iter=iter+1
                
            }
            
        }
        
    }   
    
}
overall_results=rbindlist(result)
summarized_results=overall_results[,list(mean_acc=mean(acc),sdev_acc=sd(acc),result_count=.N),by=list(approach,k)]
summarized_results[order(-mean_acc)]




dist_euc=as.matrix(dist(paa_rep5[,2:ncol(paa_rep5)]))
diag(dist_euc)=large_number
fwrite(dist_euc,sprintf('%sTRACE_euc_paa5_dist.csv',"C:/Users/mirac/HW3/Trace/Distances/Paa5/"),col.names=F)
dist_dtw=as.matrix(dtwDist(paa_rep5[,2:ncol(paa_rep5)]))
diag(dist_dtw)=large_number
fwrite(dist_dtw,sprintf('%sTRACE_dtw_paa5_dist.csv',"C:/Users/mirac/HW3/Trace/Distances/Paa5/"),col.names=F)
dist_lcss=as.matrix(TSDatabaseDistances(as.matrix(paa_rep5[,2:ncol(paa_rep5)]),distance='lcss',epsilon=0.1))
diag(dist_lcss)=large_number
fwrite(dist_lcss,sprintf('%sTRACE_lcss_paa5_epsilon_005.csv',"C:/Users/mirac/HW3/Trace/Distances/Paa5/"),col.names=F)
dist_erp=as.matrix(TSDatabaseDistances(as.matrix(paa_rep5[,2:ncol(paa_rep5)]),distance='erp',g=0.5))
diag(dist_erp)=large_number
fwrite(dist_erp,sprintf('%sTRACE_erp_paa5_gap_005.csv',"C:/Users/mirac/HW3/Trace/Distances/Paa5/"),col.names=F)


set.seed(482)
nof_rep=5
n_fold=10
cv_indices=generateCVRuns(trainclass_trace, ntimes =nof_rep, nfold = n_fold, leaveOneOut = FALSE, stratified = TRUE)
dist_files=list.files("C:/Users/mirac/HW3/Trace/Distances/Paa5/", full.names=T)
k_levels=c(1,3,5)
approach_file=list.files("C:/Users/mirac/HW3/Trace/Distances/Paa5/")
result=vector('list',length(dist_files)*nof_rep*n_fold*length(k_levels))
iter=1
for(m in 1:length(dist_files)){ 

    dist_mat=as.matrix(fread(dist_files[m],header=FALSE))
    for(i in 1:nof_rep){
        this_fold=cv_indices[[i]]
        for(j in 1:n_fold){
            test_indices=this_fold[[j]]
            for(k in 1:length(k_levels)){
                current_k=k_levels[k]
                current_fold=nn_classify_cv(dist_mat,trainclass_trace,test_indices,k=current_k)
                accuracy=sum(trainclass_trace[test_indices]==current_fold$prediction$predicted)/length(test_indices)
                tmp=data.table(approach=approach_file[m],repid=i,foldid=j,
                               k=current_k,acc=accuracy)
                result[[iter]]=tmp
                iter=iter+1
                
            }
            
        }
        
    }   
    
}
overall_results=rbindlist(result)
summarized_results=overall_results[,list(mean_acc=mean(acc),sdev_acc=sd(acc),result_count=.N),by=list(approach,k)]
summarized_results[order(-mean_acc)]


x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table


approach,k,mean_acc,sdev_acc,result_count
TRACE_dtw_paa4_dist.csv,5,0.852,0.10924602,50
TRACE_dtw_paa4_dist.csv,3,0.844,0.11277574,50
TRACE_erp_paa4_gap_005.csv,3,0.824,0.12047525,50
TRACE_dtw_paa4_dist.csv,1,0.784,0.12992934,50
TRACE_erp_paa4_gap_005.csv,1,0.784,0.11842504,50
TRACE_euc_paa4_dist.csv,1,0.760,0.11780302,50
TRACE_erp_paa4_gap_005.csv,5,0.756,0.10528872,50
TRACE_euc_paa4_dist.csv,3,0.646,0.14457538,50
TRACE_euc_paa4_dist.csv,5,0.620,0.17956864,50
TRACE_lcss_paa4_epsilon_005.csv,1,0.262,0.11408912,50


x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table


approach,k,mean_acc,sdev_acc,result_count
TRACE_dtw_paa5_dist.csv,5,0.848,0.1216217,50
TRACE_dtw_paa5_dist.csv,3,0.816,0.1218229,50
TRACE_erp_paa5_gap_005.csv,3,0.808,0.1121952,50
TRACE_erp_paa5_gap_005.csv,1,0.766,0.1271509,50
TRACE_euc_paa5_dist.csv,1,0.752,0.1232883,50
TRACE_erp_paa5_gap_005.csv,5,0.748,0.1073617,50
TRACE_dtw_paa5_dist.csv,1,0.744,0.1090965,50
TRACE_euc_paa5_dist.csv,3,0.634,0.1479244,50
TRACE_euc_paa5_dist.csv,5,0.570,0.1876275,50
TRACE_lcss_paa5_epsilon_005.csv,5,0.300,0.1049781,50


### Evaluating Piecewise Linear Approximation with important point 5 and 10

In [8]:
dist_euc=as.matrix(dist(pla_rep5[,2:ncol(pla_rep5)]))
diag(dist_euc)=large_number
fwrite(dist_euc,sprintf('%sTRACE_euc_pla5_dist.csv',"C:/Users/mirac/HW3/Trace/Distances/Pla5/"),col.names=F)
dist_dtw=as.matrix(dtwDist(pla_rep5[,2:ncol(pla_rep5)]))
diag(dist_dtw)=large_number
fwrite(dist_dtw,sprintf('%sTRACE_dtw_pla5_dist.csv',"C:/Users/mirac/HW3/Trace/Distances/Pla5/"),col.names=F)
dist_lcss=as.matrix(TSDatabaseDistances(as.matrix(pla_rep5[,2:ncol(pla_rep5)]),distance='lcss',epsilon=0.1))
diag(dist_lcss)=large_number
fwrite(dist_lcss,sprintf('%sTRACE_lcss_pla5_epsilon_005.csv',"C:/Users/mirac/HW3/Trace/Distances/Pla5/"),col.names=F)
dist_erp=as.matrix(TSDatabaseDistances(as.matrix(pla_rep5[,2:ncol(pla_rep5)]),distance='erp',g=0.5))
diag(dist_erp)=large_number
fwrite(dist_erp,sprintf('%sTRACE_erp_pla5_gap_005.csv',"C:/Users/mirac/HW3/Trace/Distances/Pla5/"),col.names=F)


set.seed(482)
nof_rep=5
n_fold=10
cv_indices=generateCVRuns(trainclass_trace, ntimes =nof_rep, nfold = n_fold, leaveOneOut = FALSE, stratified = TRUE)
dist_files=list.files("C:/Users/mirac/HW3/Trace/Distances/Pla5/", full.names=T)
k_levels=c(1,3,5)
approach_file=list.files("C:/Users/mirac/HW3/Trace/Distances/Pla5/")
result=vector('list',length(dist_files)*nof_rep*n_fold*length(k_levels))
iter=1
for(m in 1:length(dist_files)){ 

    dist_mat=as.matrix(fread(dist_files[m],header=FALSE))
    for(i in 1:nof_rep){
        this_fold=cv_indices[[i]]
        for(j in 1:n_fold){
            test_indices=this_fold[[j]]
            for(k in 1:length(k_levels)){
                current_k=k_levels[k]
                current_fold=nn_classify_cv(dist_mat,trainclass_trace,test_indices,k=current_k)
                accuracy=sum(trainclass_trace[test_indices]==current_fold$prediction$predicted)/length(test_indices)
                tmp=data.table(approach=approach_file[m],repid=i,foldid=j,
                               k=current_k,acc=accuracy)
                result[[iter]]=tmp
                iter=iter+1
                
            }
            
        }
        
    }   
    
}
overall_results=rbindlist(result)
summarized_results=overall_results[,list(mean_acc=mean(acc),sdev_acc=sd(acc),result_count=.N),by=list(approach,k)]
summarized_results[order(-mean_acc)]




dist_euc=as.matrix(dist(pla_rep10[,2:ncol(pla_rep10)]))
diag(dist_euc)=large_number
fwrite(dist_euc,sprintf('%sTRACE_euc_pla10_dist.csv',"C:/Users/mirac/HW3/Trace/Distances/Pla10/"),col.names=F)
dist_dtw=as.matrix(dtwDist(pla_rep10[,2:ncol(pla_rep10)]))
diag(dist_dtw)=large_number
fwrite(dist_dtw,sprintf('%sTRACE_dtw_pla10_dist.csv',"C:/Users/mirac/HW3/Trace/Distances/Pla10/"),col.names=F)
dist_lcss=as.matrix(TSDatabaseDistances(as.matrix(pla_rep10[,2:ncol(pla_rep10)]),distance='lcss',epsilon=0.1))
diag(dist_lcss)=large_number
fwrite(dist_lcss,sprintf('%sTRACE_lcss_pla10_epsilon_005.csv',"C:/Users/mirac/HW3/Trace/Distances/Pla10/"),col.names=F)
dist_erp=as.matrix(TSDatabaseDistances(as.matrix(pla_rep10[,2:ncol(pla_rep10)]),distance='erp',g=0.5))
diag(dist_erp)=large_number
fwrite(dist_erp,sprintf('%sTRACE_erp_pla10_gap_005.csv',"C:/Users/mirac/HW3/Trace/Distances/Pla10/"),col.names=F)


set.seed(482)
nof_rep=5
n_fold=10
cv_indices=generateCVRuns(trainclass_trace, ntimes =nof_rep, nfold = n_fold, leaveOneOut = FALSE, stratified = TRUE)
dist_files=list.files("C:/Users/mirac/HW3/Trace/Distances/Pla10/", full.names=T)
k_levels=c(1,3,5)
approach_file=list.files("C:/Users/mirac/HW3/Trace/Distances/Pla10/")
result=vector('list',length(dist_files)*nof_rep*n_fold*length(k_levels))
iter=1
for(m in 1:length(dist_files)){ 

    dist_mat=as.matrix(fread(dist_files[m],header=FALSE))
    for(i in 1:nof_rep){
        this_fold=cv_indices[[i]]
        for(j in 1:n_fold){
            test_indices=this_fold[[j]]
            for(k in 1:length(k_levels)){
                current_k=k_levels[k]
                current_fold=nn_classify_cv(dist_mat,trainclass_trace,test_indices,k=current_k)
                accuracy=sum(trainclass_trace[test_indices]==current_fold$prediction$predicted)/length(test_indices)
                tmp=data.table(approach=approach_file[m],repid=i,foldid=j,
                               k=current_k,acc=accuracy)
                result[[iter]]=tmp
                iter=iter+1
                
            }
            
        }
        
    }   
    
}
overall_results=rbindlist(result)
summarized_results=overall_results[,list(mean_acc=mean(acc),sdev_acc=sd(acc),result_count=.N),by=list(approach,k)]
summarized_results[order(-mean_acc)]


x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table


approach,k,mean_acc,sdev_acc,result_count
TRACE_erp_pla5_gap_005.csv,5,0.790,0.09313146,50
TRACE_dtw_pla5_dist.csv,3,0.778,0.10159945,50
TRACE_dtw_pla5_dist.csv,5,0.766,0.11177602,50
TRACE_dtw_pla5_dist.csv,1,0.752,0.11292042,50
TRACE_erp_pla5_gap_005.csv,3,0.744,0.12315578,50
TRACE_euc_pla5_dist.csv,3,0.722,0.13292625,50
TRACE_erp_pla5_gap_005.csv,1,0.720,0.11428571,50
TRACE_euc_pla5_dist.csv,1,0.694,0.13463480,50
TRACE_euc_pla5_dist.csv,5,0.656,0.11633870,50
TRACE_lcss_pla5_epsilon_005.csv,1,0.246,0.11816626,50


x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table


approach,k,mean_acc,sdev_acc,result_count
TRACE_dtw_pla10_dist.csv,5,0.874,0.09435063,50
TRACE_dtw_pla10_dist.csv,3,0.866,0.09171829,50
TRACE_dtw_pla10_dist.csv,1,0.804,0.09467452,50
TRACE_erp_pla10_gap_005.csv,1,0.790,0.11823532,50
TRACE_euc_pla10_dist.csv,1,0.766,0.13644166,50
TRACE_erp_pla10_gap_005.csv,3,0.764,0.12249115,50
TRACE_erp_pla10_gap_005.csv,5,0.730,0.11994897,50
TRACE_euc_pla10_dist.csv,3,0.638,0.16646382,50
TRACE_euc_pla10_dist.csv,5,0.616,0.15301060,50
TRACE_lcss_pla10_epsilon_005.csv,3,0.208,0.11036119,50


### According to Evaluations the Best Combination for Trace Data is : Raw Erp with g=0.5 k=5
##### Reading Test Data and Manipulation

In [9]:
testdata_trace=fread("C:/Users/mirac/HW3/Trace/Trace_TEST.txt")
setnames(testdata_trace,'V1','class')
testdata_trace=testdata_trace[order(class)]
testdata_trace[,class:=as.character(class)]
testdata_trace[,id:=1:.N]
longtest_trace=melt(testdata_trace,id.vars=c('id','class'))
longtest_trace[,time:=as.numeric(gsub("\\D", "", variable))-1]
longtest_trace=longtest_trace[,list(id,class,time,value)]
longtest_trace=longtest_trace[order(id,time)]


### Obtaining Accuracy Rate on Test Data
#### It is 95%

In [10]:
testclass_trace=as.matrix(testdata_trace[,1])

dist_erp_test=as.matrix(TSDatabaseDistances(as.matrix(testdata_trace[,2:ncol(testdata_trace)]),distance='erp',g=0.5))
diag(dist_erp_test)=large_number


neighborhood=apply(dist_erp_test,1,order)
nearest_class=apply(neighborhood[1:5,],2,function(x) {testclass_trace[x]})
nearest_class=data.table(id=1:ncol(neighborhood),t(nearest_class))
long_nn_class=melt(nearest_class,'id')
long_nn_class=long_nn_class[order(id)]
class_counts=long_nn_class[,.N,list(id,value)]
class_predictions=class_counts[,list(predicted=value[which.max(N)]),by=list(id)]
acc=sum(testclass_trace==class_predictions$predicted)/length(testclass_trace)
acc

[1] 0.95

## Same Evaluation Steps for SmoothSubspace Data 
#### With Exactly Same Parameters as stated above

In [11]:
traindata_smooth=fread("C:/Users/mirac/HW3/SmoothSubspace/SmoothSubspace_TRAIN.txt")
 setnames(traindata_smooth,'V1','class')
 traindata_smooth=traindata_smooth[order(class)]
 traindata_smooth[,class:=as.character(class)]
 traindata_smooth[,id:=1:.N]
 longtrain_smooth=melt(traindata_smooth,id.vars=c('id','class'))
longtrain_smooth[,time:=as.numeric(gsub("\\D", "", variable))-1]
longtrain_smooth=longtrain_smooth[,list(id,class,time,value)]
longtrain_smooth=longtrain_smooth[order(id,time)]

In [12]:
reprfunc=function(x){
    selected_series=x
    data_plot=longtrain_smooth[id==x]
    
    
    segment_length=5
    longtrain_smooth=longtrain_smooth[order(id,time)]
    data_ts=longtrain_smooth[id==x]$value
    paa_rep5=repr_paa(data_ts, segment_length, meanC)
    
    dummy_time=c(1:(length(paa_rep5)-1))*segment_length 
    dummy_time=c(dummy_time,nrow(data_plot)) 
    
    dt_paa=data.table(time=dummy_time,paa_rep5=paa_rep5)
    data_plot=merge(data_plot,dt_paa,by='time',all.x=T)
    data_plot[,paa_rep5:=nafill(paa_rep5,'nocb')]
    
    
    segment_length=4
    longtrain_smooth=longtrain_smooth[order(id,time)]
    data_ts=longtrain_smooth[id==x]$value
    paa_rep4=repr_paa(data_ts, segment_length, meanC)
    
    dummy_time=c(1:(length(paa_rep4)-1))*segment_length 
    dummy_time=c(dummy_time,nrow(data_plot)) 
    
    dt_paa=data.table(time=dummy_time,paa_rep4=paa_rep4)
    data_plot=merge(data_plot,dt_paa,by='time',all.x=T)
    data_plot[,paa_rep4:=nafill(paa_rep4,'nocb')]
    
    
    n_important = 5
    pla_rep5=repr_pla(data_ts, times = n_important, return = "both")
    setnames(pla_rep5,c('places','points'),c('time','pla_rep5'))
    pla_rep5$time=pla_rep5$time+1
    
    data_plot=merge(data_plot,pla_rep5,by='time',all.x=T)
    data_plot[, pla_rep5 := na.approx(pla_rep5)]
    
    
    n_important = 10
    pla_rep10=repr_pla(data_ts, times = n_important, return = "both")
    setnames(pla_rep10,c('places','points'),c('time','pla_rep10'))
    pla_rep10$time=pla_rep10$time+1
    
    data_plot=merge(data_plot,pla_rep10,by='time',all.x=T)
    data_plot[, pla_rep10 := na.approx(pla_rep10)]
    return(data_plot)
}


representations=data.table()

for(i in 1:150){
    representations=rbind(representations,reprfunc(i))
}


paa_rep5=dcast(representations,id~time,value.var='paa_rep5')
paa_rep4=dcast(representations,id~time,value.var='paa_rep4')
pla_rep5=dcast(representations,id~time,value.var='pla_rep5')
pla_rep10=dcast(representations,id~time,value.var='pla_rep10')

In [13]:
trainclass_smooth=as.matrix(traindata_smooth[,1])
large_number=10000
dist_euc=as.matrix(dist(traindata_smooth[,2:ncol(traindata_smooth)]))
diag(dist_euc)=large_number
fwrite(dist_euc,sprintf('%sSMOOTH_euc_raw_dist.csv',"C:/Users/mirac/HW3/SmoothSubspace/Distances/Raw/"),col.names=F)
dist_dtw=as.matrix(dtwDist(traindata_smooth[,2:ncol(traindata_smooth)]))
diag(dist_dtw)=large_number
fwrite(dist_dtw,sprintf('%sSMOOTH_dtw_raw_dist.csv',"C:/Users/mirac/HW3/SmoothSubspace/Distances/Raw/"),col.names=F)
dist_lcss=as.matrix(TSDatabaseDistances(as.matrix(traindata_smooth[,2:ncol(traindata_smooth)]),distance='lcss',epsilon=0.1))
diag(dist_lcss)=large_number
fwrite(dist_lcss,sprintf('%sSMOOTH_lcss_raw_epsilon_005.csv',"C:/Users/mirac/HW3/SmoothSubspace/Distances/Raw/"),col.names=F)
dist_erp=as.matrix(TSDatabaseDistances(as.matrix(traindata_smooth[,2:ncol(traindata_smooth)]),distance='erp',g=0.5))
diag(dist_erp)=large_number
fwrite(dist_erp,sprintf('%sSMOOTH_erp_raw_gap_005.csv',"C:/Users/mirac/HW3/SmoothSubspace/Distances/Raw/"),col.names=F)



nn_classify_cv=function(dist_matrix,train_class,test_indices,k=1){
    
    test_distances_to_train=dist_matrix[test_indices,]
    test_distances_to_train=test_distances_to_train[,-test_indices]
    train_class=train_class[-test_indices]
    ordered_indices=apply(test_distances_to_train,1,order)
    if(k==1){
        nearest_class=as.numeric(trainclass_smooth[as.numeric(ordered_indices[1,])])
        nearest_class=data.table(id=test_indices,nearest_class)
    } else {
        nearest_class=apply(ordered_indices[1:k,],2,function(x) {trainclass_smooth[x]})
        nearest_class=data.table(id=test_indices,t(nearest_class))
    }
    
    long_nn_class=melt(nearest_class,'id')
    
    class_counts=long_nn_class[,.N,list(id,value)]
    class_counts[,predicted_prob:=N/k]
    wide_class_prob_predictions=dcast(class_counts,id~value,value.var='predicted_prob')
    wide_class_prob_predictions[is.na(wide_class_prob_predictions)]=0
    class_predictions=class_counts[,list(predicted=value[which.max(N)]),by=list(id)]
    
    
    return(list(prediction=class_predictions,prob_estimates=wide_class_prob_predictions))
    
}



set.seed(482)
nof_rep=5
n_fold=10
cv_indices=generateCVRuns(trainclass_smooth, ntimes =nof_rep, nfold = n_fold, leaveOneOut = FALSE, stratified = TRUE)
dist_files=list.files("C:/Users/mirac/HW3/SmoothSubspace/Distances/Raw/", full.names=T)
k_levels=c(1,3,5)
approach_file=list.files("C:/Users/mirac/HW3/SmoothSubspace/Distances/Raw/")
result=vector('list',length(dist_files)*nof_rep*n_fold*length(k_levels))
iter=1
for(m in 1:length(dist_files)){ 

    dist_mat=as.matrix(fread(dist_files[m],header=FALSE))
    for(i in 1:nof_rep){
        this_fold=cv_indices[[i]]
        for(j in 1:n_fold){
            test_indices=this_fold[[j]]
            for(k in 1:length(k_levels)){
                current_k=k_levels[k]
                current_fold=nn_classify_cv(dist_mat,trainclass_smooth,test_indices,k=current_k)
                accuracy=sum(trainclass_smooth[test_indices]==current_fold$prediction$predicted)/length(test_indices)
                tmp=data.table(approach=approach_file[m],repid=i,foldid=j,
                               k=current_k,acc=accuracy)
                result[[iter]]=tmp
                iter=iter+1
                
            }
            
        }
        
    }   
    
}
overall_results=rbindlist(result)
summarized_results=overall_results[,list(mean_acc=mean(acc),sdev_acc=sd(acc),result_count=.N),by=list(approach,k)]
summarized_results[order(-mean_acc)]

x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table


approach,k,mean_acc,sdev_acc,result_count
SMOOTH_erp_raw_gap_005.csv,1,0.88933333,0.07572477,50
SMOOTH_erp_raw_gap_005.csv,3,0.88933333,0.07572477,50
SMOOTH_dtw_raw_dist.csv,1,0.88400000,0.07596395,50
SMOOTH_dtw_raw_dist.csv,3,0.88400000,0.07596395,50
SMOOTH_euc_raw_dist.csv,1,0.88400000,0.07596395,50
SMOOTH_dtw_raw_dist.csv,5,0.88266667,0.07322812,50
SMOOTH_erp_raw_gap_005.csv,5,0.88266667,0.07322812,50
SMOOTH_euc_raw_dist.csv,5,0.88266667,0.07322812,50
SMOOTH_euc_raw_dist.csv,3,0.87866667,0.07700752,50
SMOOTH_lcss_raw_epsilon_005.csv,1,0.16400000,0.09056136,50


In [14]:
dist_euc=as.matrix(dist(paa_rep4[,2:ncol(paa_rep4)]))
diag(dist_euc)=large_number
fwrite(dist_euc,sprintf('%sSMOOTH_euc_paa4_dist.csv',"C:/Users/mirac/HW3/SmoothSubspace/Distances/Paa4/"),col.names=F)
dist_dtw=as.matrix(dtwDist(paa_rep4[,2:ncol(paa_rep4)]))
diag(dist_dtw)=large_number
fwrite(dist_dtw,sprintf('%sSMOOTH_dtw_paa4_dist.csv',"C:/Users/mirac/HW3/SmoothSubspace/Distances/Paa4/"),col.names=F)
dist_lcss=as.matrix(TSDatabaseDistances(as.matrix(paa_rep4[,2:ncol(paa_rep4)]),distance='lcss',epsilon=0.1))
diag(dist_lcss)=large_number
fwrite(dist_lcss,sprintf('%sSMOOTH_lcss_paa4_epsilon_005.csv',"C:/Users/mirac/HW3/SmoothSubspace/Distances/Paa4/"),col.names=F)
dist_erp=as.matrix(TSDatabaseDistances(as.matrix(paa_rep4[,2:ncol(paa_rep4)]),distance='erp',g=0.5))
diag(dist_erp)=large_number
fwrite(dist_erp,sprintf('%sSMOOTH_erp_paa4_gap_005.csv',"C:/Users/mirac/HW3/SmoothSubspace/Distances/Paa4/"),col.names=F)

set.seed(482)
nof_rep=5
n_fold=10
cv_indices=generateCVRuns(trainclass_smooth, ntimes =nof_rep, nfold = n_fold, leaveOneOut = FALSE, stratified = TRUE)
dist_files=list.files("C:/Users/mirac/HW3/SmoothSubspace/Distances/Paa4/", full.names=T)
k_levels=c(1,3,5)
approach_file=list.files("C:/Users/mirac/HW3/SmoothSubspace/Distances/Paa4/")
result=vector('list',length(dist_files)*nof_rep*n_fold*length(k_levels))
iter=1
for(m in 1:length(dist_files)){ 

    dist_mat=as.matrix(fread(dist_files[m],header=FALSE))
    for(i in 1:nof_rep){
        this_fold=cv_indices[[i]]
        for(j in 1:n_fold){
            test_indices=this_fold[[j]]
            for(k in 1:length(k_levels)){
                current_k=k_levels[k]
                current_fold=nn_classify_cv(dist_mat,trainclass_smooth,test_indices,k=current_k)
                accuracy=sum(trainclass_smooth[test_indices]==current_fold$prediction$predicted)/length(test_indices)
                tmp=data.table(approach=approach_file[m],repid=i,foldid=j,
                               k=current_k,acc=accuracy)
                result[[iter]]=tmp
                iter=iter+1
                
            }
            
        }
        
    }   
    
}
overall_results=rbindlist(result)
summarized_results=overall_results[,list(mean_acc=mean(acc),sdev_acc=sd(acc),result_count=.N),by=list(approach,k)]
summarized_results[order(-mean_acc)]




dist_euc=as.matrix(dist(paa_rep5[,2:ncol(paa_rep5)]))
diag(dist_euc)=large_number
fwrite(dist_euc,sprintf('%sSMOOTH_euc_paa5_dist.csv',"C:/Users/mirac/HW3/SmoothSubspace/Distances/Paa5/"),col.names=F)
dist_dtw=as.matrix(dtwDist(paa_rep5[,2:ncol(paa_rep5)]))
diag(dist_dtw)=large_number
fwrite(dist_dtw,sprintf('%sSMOOTH_dtw_paa5_dist.csv',"C:/Users/mirac/HW3/SmoothSubspace/Distances/Paa5/"),col.names=F)
dist_lcss=as.matrix(TSDatabaseDistances(as.matrix(paa_rep5[,2:ncol(paa_rep5)]),distance='lcss',epsilon=0.1))
diag(dist_lcss)=large_number
fwrite(dist_lcss,sprintf('%sSMOOTH_lcss_paa5_epsilon_005.csv',"C:/Users/mirac/HW3/SmoothSubspace/Distances/Paa5/"),col.names=F)
dist_erp=as.matrix(TSDatabaseDistances(as.matrix(paa_rep5[,2:ncol(paa_rep5)]),distance='erp',g=0.5))
diag(dist_erp)=large_number
fwrite(dist_erp,sprintf('%sSMOOTH_erp_paa5_gap_005.csv',"C:/Users/mirac/HW3/SmoothSubspace/Distances/Paa5/"),col.names=F)


set.seed(482)
nof_rep=5
n_fold=10
cv_indices=generateCVRuns(trainclass_smooth, ntimes =nof_rep, nfold = n_fold, leaveOneOut = FALSE, stratified = TRUE)
dist_files=list.files("C:/Users/mirac/HW3/SmoothSubspace/Distances/Paa5/", full.names=T)
k_levels=c(1,3,5)
approach_file=list.files("C:/Users/mirac/HW3/SmoothSubspace/Distances/Paa5/")
result=vector('list',length(dist_files)*nof_rep*n_fold*length(k_levels))
iter=1
for(m in 1:length(dist_files)){ 

    dist_mat=as.matrix(fread(dist_files[m],header=FALSE))
    for(i in 1:nof_rep){
        this_fold=cv_indices[[i]]
        for(j in 1:n_fold){
            test_indices=this_fold[[j]]
            for(k in 1:length(k_levels)){
                current_k=k_levels[k]
                current_fold=nn_classify_cv(dist_mat,trainclass_smooth,test_indices,k=current_k)
                accuracy=sum(trainclass_smooth[test_indices]==current_fold$prediction$predicted)/length(test_indices)
                tmp=data.table(approach=approach_file[m],repid=i,foldid=j,
                               k=current_k,acc=accuracy)
                result[[iter]]=tmp
                iter=iter+1
                
            }
            
        }
        
    }   
    
}
overall_results=rbindlist(result)
summarized_results=overall_results[,list(mean_acc=mean(acc),sdev_acc=sd(acc),result_count=.N),by=list(approach,k)]
summarized_results[order(-mean_acc)]

x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table


approach,k,mean_acc,sdev_acc,result_count
SMOOTH_euc_paa4_dist.csv,3,0.8373333,0.10271595,50
SMOOTH_erp_paa4_gap_005.csv,3,0.8320000,0.08219219,50
SMOOTH_erp_paa4_gap_005.csv,5,0.8280000,0.07978429,50
SMOOTH_euc_paa4_dist.csv,5,0.8066667,0.09452111,50
SMOOTH_dtw_paa4_dist.csv,5,0.8040000,0.08771217,50
SMOOTH_erp_paa4_gap_005.csv,1,0.7986667,0.08675558,50
SMOOTH_euc_paa4_dist.csv,1,0.7973333,0.09801661,50
SMOOTH_dtw_paa4_dist.csv,3,0.7973333,0.09614803,50
SMOOTH_dtw_paa4_dist.csv,1,0.7333333,0.09897433,50
SMOOTH_lcss_paa4_epsilon_005.csv,1,0.2080000,0.10046603,50


x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table


approach,k,mean_acc,sdev_acc,result_count
SMOOTH_dtw_paa5_dist.csv,5,0.9320000,0.06665306,50
SMOOTH_euc_paa5_dist.csv,5,0.9226667,0.06224813,50
SMOOTH_erp_paa5_gap_005.csv,5,0.9146667,0.07382028,50
SMOOTH_dtw_paa5_dist.csv,3,0.9080000,0.07955659,50
SMOOTH_erp_paa5_gap_005.csv,3,0.9000000,0.08438117,50
SMOOTH_euc_paa5_dist.csv,3,0.8946667,0.07509933,50
SMOOTH_dtw_paa5_dist.csv,1,0.8626667,0.08455298,50
SMOOTH_euc_paa5_dist.csv,1,0.8480000,0.07972743,50
SMOOTH_erp_paa5_gap_005.csv,1,0.8160000,0.09948621,50
SMOOTH_lcss_paa5_epsilon_005.csv,5,0.2560000,0.08121526,50


In [15]:
dist_euc=as.matrix(dist(pla_rep5[,2:ncol(pla_rep5)]))
diag(dist_euc)=large_number
fwrite(dist_euc,sprintf('%sSMOOTH_euc_pla5_dist.csv',"C:/Users/mirac/HW3/SmoothSubspace/Distances/Pla5/"),col.names=F)
dist_dtw=as.matrix(dtwDist(pla_rep5[,2:ncol(pla_rep5)]))
diag(dist_dtw)=large_number
fwrite(dist_dtw,sprintf('%sSMOOTH_dtw_pla5_dist.csv',"C:/Users/mirac/HW3/SmoothSubspace/Distances/Pla5/"),col.names=F)
dist_lcss=as.matrix(TSDatabaseDistances(as.matrix(pla_rep5[,2:ncol(pla_rep5)]),distance='lcss',epsilon=0.1))
diag(dist_lcss)=large_number
fwrite(dist_lcss,sprintf('%sSMOOTH_lcss_pla5_epsilon_005.csv',"C:/Users/mirac/HW3/SmoothSubspace/Distances/Pla5/"),col.names=F)
dist_erp=as.matrix(TSDatabaseDistances(as.matrix(pla_rep5[,2:ncol(pla_rep5)]),distance='erp',g=0.5))
diag(dist_erp)=large_number
fwrite(dist_erp,sprintf('%sSMOOTH_erp_pla5_gap_005.csv',"C:/Users/mirac/HW3/SmoothSubspace/Distances/Pla5/"),col.names=F)


set.seed(482)
nof_rep=5
n_fold=10
cv_indices=generateCVRuns(trainclass_smooth, ntimes =nof_rep, nfold = n_fold, leaveOneOut = FALSE, stratified = TRUE)
dist_files=list.files("C:/Users/mirac/HW3/SmoothSubspace/Distances/Pla5/", full.names=T)
k_levels=c(1,3,5)
approach_file=list.files("C:/Users/mirac/HW3/SmoothSubspace/Distances/Pla5/")
result=vector('list',length(dist_files)*nof_rep*n_fold*length(k_levels))
iter=1
for(m in 1:length(dist_files)){ 

    dist_mat=as.matrix(fread(dist_files[m],header=FALSE))
    for(i in 1:nof_rep){
        this_fold=cv_indices[[i]]
        for(j in 1:n_fold){
            test_indices=this_fold[[j]]
            for(k in 1:length(k_levels)){
                current_k=k_levels[k]
                current_fold=nn_classify_cv(dist_mat,trainclass_smooth,test_indices,k=current_k)
                accuracy=sum(trainclass_smooth[test_indices]==current_fold$prediction$predicted)/length(test_indices)
                tmp=data.table(approach=approach_file[m],repid=i,foldid=j,
                               k=current_k,acc=accuracy)
                result[[iter]]=tmp
                iter=iter+1
                
            }
            
        }
        
    }   
    
}
overall_results=rbindlist(result)
summarized_results=overall_results[,list(mean_acc=mean(acc),sdev_acc=sd(acc),result_count=.N),by=list(approach,k)]
summarized_results[order(-mean_acc)]




dist_euc=as.matrix(dist(pla_rep10[,2:ncol(pla_rep10)]))
diag(dist_euc)=large_number
fwrite(dist_euc,sprintf('%sSMOOTH_euc_pla10_dist.csv',"C:/Users/mirac/HW3/SmoothSubspace/Distances/Pla10/"),col.names=F)
dist_dtw=as.matrix(dtwDist(pla_rep10[,2:ncol(pla_rep10)]))
diag(dist_dtw)=large_number
fwrite(dist_dtw,sprintf('%sSMOOTH_dtw_pla10_dist.csv',"C:/Users/mirac/HW3/SmoothSubspace/Distances/Pla10/"),col.names=F)
dist_lcss=as.matrix(TSDatabaseDistances(as.matrix(pla_rep10[,2:ncol(pla_rep10)]),distance='lcss',epsilon=0.1))
diag(dist_lcss)=large_number
fwrite(dist_lcss,sprintf('%sSMOOTH_lcss_pla10_epsilon_005.csv',"C:/Users/mirac/HW3/SmoothSubspace/Distances/Pla10/"),col.names=F)
dist_erp=as.matrix(TSDatabaseDistances(as.matrix(pla_rep10[,2:ncol(pla_rep10)]),distance='erp',g=0.5))
diag(dist_erp)=large_number
fwrite(dist_erp,sprintf('%sSMOOTH_erp_pla10_gap_005.csv',"C:/Users/mirac/HW3/SmoothSubspace/Distances/Pla10/"),col.names=F)


set.seed(482)
nof_rep=5
n_fold=10
cv_indices=generateCVRuns(trainclass_smooth, ntimes =nof_rep, nfold = n_fold, leaveOneOut = FALSE, stratified = TRUE)
dist_files=list.files("C:/Users/mirac/HW3/SmoothSubspace/Distances/Pla10/", full.names=T)
k_levels=c(1,3,5)
approach_file=list.files("C:/Users/mirac/HW3/SmoothSubspace/Distances/Pla10/")
result=vector('list',length(dist_files)*nof_rep*n_fold*length(k_levels))
iter=1
for(m in 1:length(dist_files)){ 

    dist_mat=as.matrix(fread(dist_files[m],header=FALSE))
    for(i in 1:nof_rep){
        this_fold=cv_indices[[i]]
        for(j in 1:n_fold){
            test_indices=this_fold[[j]]
            for(k in 1:length(k_levels)){
                current_k=k_levels[k]
                current_fold=nn_classify_cv(dist_mat,trainclass_smooth,test_indices,k=current_k)
                accuracy=sum(trainclass_smooth[test_indices]==current_fold$prediction$predicted)/length(test_indices)
                tmp=data.table(approach=approach_file[m],repid=i,foldid=j,
                               k=current_k,acc=accuracy)
                result[[iter]]=tmp
                iter=iter+1
                
            }
            
        }
        
    }   
    
}
overall_results=rbindlist(result)
summarized_results=overall_results[,list(mean_acc=mean(acc),sdev_acc=sd(acc),result_count=.N),by=list(approach,k)]
summarized_results[order(-mean_acc)]



x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table


approach,k,mean_acc,sdev_acc,result_count
SMOOTH_dtw_pla5_dist.csv,5,0.7373333,0.11461065,50
SMOOTH_dtw_pla5_dist.csv,3,0.7253333,0.11557997,50
SMOOTH_erp_pla5_gap_005.csv,5,0.7040000,0.11520267,50
SMOOTH_erp_pla5_gap_005.csv,3,0.6986667,0.11368094,50
SMOOTH_erp_pla5_gap_005.csv,1,0.6906667,0.14046247,50
SMOOTH_euc_pla5_dist.csv,3,0.6733333,0.13213749,50
SMOOTH_euc_pla5_dist.csv,5,0.6706667,0.11926456,50
SMOOTH_euc_pla5_dist.csv,1,0.6666667,0.11664237,50
SMOOTH_dtw_pla5_dist.csv,1,0.6120000,0.11947732,50
SMOOTH_lcss_pla5_epsilon_005.csv,1,0.2653333,0.10123052,50


x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table


approach,k,mean_acc,sdev_acc,result_count
SMOOTH_erp_pla10_gap_005.csv,5,0.94266667,0.05872415,50
SMOOTH_erp_pla10_gap_005.csv,3,0.90400000,0.06888816,50
SMOOTH_euc_pla10_dist.csv,5,0.88800000,0.08978685,50
SMOOTH_euc_pla10_dist.csv,3,0.84133333,0.07112953,50
SMOOTH_dtw_pla10_dist.csv,5,0.83333333,0.08959470,50
SMOOTH_erp_pla10_gap_005.csv,1,0.82133333,0.08774318,50
SMOOTH_dtw_pla10_dist.csv,3,0.81066667,0.09268971,50
SMOOTH_euc_pla10_dist.csv,1,0.78800000,0.09398220,50
SMOOTH_dtw_pla10_dist.csv,1,0.76000000,0.11971243,50
SMOOTH_lcss_pla10_epsilon_005.csv,1,0.13866667,0.08712074,50


### According to Evaluations Best Parameter Combination for SmoothSubspace Data:
##### Piecewise Linear Approximation with 10 important points and Erp with g=0.5 k=5
##### So Obtaining Accuracy Rate for this Combination
#### It is 97%

In [16]:
testdata_smooth=fread("C:/Users/mirac/HW3/SmoothSubspace/SmoothSubspace_TEST.txt")
setnames(testdata_smooth,'V1','class')
testdata_smooth=testdata_smooth[order(class)]
testdata_smooth[,class:=as.character(class)]
testdata_smooth[,id:=1:.N]
longtest_smooth=melt(testdata_smooth,id.vars=c('id','class'))
longtest_smooth[,time:=as.numeric(gsub("\\D", "", variable))-1]
longtest_smooth=longtest_smooth[,list(id,class,time,value)]
longtest_smooth=longtest_smooth[order(id,time)]


reprfunc=function(x){
    selected_series=x
    data_plot=longtest_smooth[id==x]
data_ts=longtrain_smooth[id==x]$value
    
    
     n_important = 10
    pla_rep10=repr_pla(data_ts, times = n_important, return = "both")
    setnames(pla_rep10,c('places','points'),c('time','pla_rep10'))
    pla_rep10$time=pla_rep10$time+1
    
    data_plot=merge(data_plot,pla_rep10,by='time',all.x=T)
    data_plot[, pla_rep10 := na.approx(pla_rep10)]
    
   
    return(data_plot)
}


representations=data.table()

for(i in 1:150){
    representations=rbind(representations,reprfunc(i))
}

representations=dcast(representations,id~time,value.var = 'pla_rep10')


testclass_smooth=as.matrix(testdata_smooth[,1])

dist_erp_test=as.matrix(TSDatabaseDistances(as.matrix(representations[,2:ncol(representations)]),distance='erp',g=0.5))
diag(dist_erp_test)=large_number


neighborhood=apply(dist_erp_test,1,order)
nearest_class=apply(neighborhood[1:5,],2,function(x) {testclass_smooth[x]})
nearest_class=data.table(id=1:ncol(neighborhood),t(nearest_class))
long_nn_class=melt(nearest_class,'id')
long_nn_class=long_nn_class[order(id)]
class_counts=long_nn_class[,.N,list(id,value)]
class_predictions=class_counts[,list(predicted=value[which.max(N)]),by=list(id)]
acc=sum(testclass_smooth==class_predictions$predicted)/length(testclass_smooth)
acc

[1] 0.9666667

## Same Steps for Plane Dataset

In [17]:
traindata_plane=fread("C:/Users/mirac/HW3/Plane/Plane_TRAIN.txt")
 setnames(traindata_plane,'V1','class')
 traindata_plane=traindata_plane[order(class)]
 traindata_plane[,class:=as.character(class)]
 traindata_plane[,id:=1:.N]
 longtrain_plane=melt(traindata_plane,id.vars=c('id','class'))
longtrain_plane[,time:=as.numeric(gsub("\\D", "", variable))-1]
longtrain_plane=longtrain_plane[,list(id,class,time,value)]
longtrain_plane=longtrain_plane[order(id,time)]





reprfunc=function(x){
    selected_series=x
    data_plot=longtrain_plane[id==x]
    
    
    segment_length=5
    longtrain_plane=longtrain_plane[order(id,time)]
    data_ts=longtrain_plane[id==x]$value
    paa_rep5=repr_paa(data_ts, segment_length, meanC)
    
    dummy_time=c(1:(length(paa_rep5)-1))*segment_length 
    dummy_time=c(dummy_time,nrow(data_plot)) 
    
    dt_paa=data.table(time=dummy_time,paa_rep5=paa_rep5)
    data_plot=merge(data_plot,dt_paa,by='time',all.x=T)
    data_plot[,paa_rep5:=nafill(paa_rep5,'nocb')]
    
    
    segment_length=4
    longtrain_plane=longtrain_plane[order(id,time)]
    data_ts=longtrain_plane[id==x]$value
    paa_rep4=repr_paa(data_ts, segment_length, meanC)
    
    dummy_time=c(1:(length(paa_rep4)-1))*segment_length 
    dummy_time=c(dummy_time,nrow(data_plot)) 
    
    dt_paa=data.table(time=dummy_time,paa_rep4=paa_rep4)
    data_plot=merge(data_plot,dt_paa,by='time',all.x=T)
    data_plot[,paa_rep4:=nafill(paa_rep4,'nocb')]
    
    
    n_important = 5
    pla_rep5=repr_pla(data_ts, times = n_important, return = "both")
    setnames(pla_rep5,c('places','points'),c('time','pla_rep5'))
    pla_rep5$time=pla_rep5$time+1
    
    data_plot=merge(data_plot,pla_rep5,by='time',all.x=T)
    data_plot[, pla_rep5 := na.approx(pla_rep5)]
    
    
    n_important = 10
    pla_rep10=repr_pla(data_ts, times = n_important, return = "both")
    setnames(pla_rep10,c('places','points'),c('time','pla_rep10'))
    pla_rep10$time=pla_rep10$time+1
    
    data_plot=merge(data_plot,pla_rep10,by='time',all.x=T)
    data_plot[, pla_rep10 := na.approx(pla_rep10)]
    return(data_plot)
}


representations=data.table()

for(i in 1:105){
    representations=rbind(representations,reprfunc(i))
}


paa_rep5=dcast(representations,id~time,value.var='paa_rep5')
paa_rep4=dcast(representations,id~time,value.var='paa_rep4')
pla_rep5=dcast(representations,id~time,value.var='pla_rep5')
pla_rep10=dcast(representations,id~time,value.var='pla_rep10')






trainclass_plane=as.matrix(traindata_plane[,1])
large_number=10000
dist_euc=as.matrix(dist(traindata_plane[,2:ncol(traindata_plane)]))
diag(dist_euc)=large_number
fwrite(dist_euc,sprintf('%sPLANE_euc_raw_dist.csv',"C:/Users/mirac/HW3/Plane/Distances/Raw/"),col.names=F)
dist_dtw=as.matrix(dtwDist(traindata_plane[,2:ncol(traindata_plane)]))
diag(dist_dtw)=large_number
fwrite(dist_dtw,sprintf('%sPLANE_dtw_raw_dist.csv',"C:/Users/mirac/HW3/Plane/Distances/Raw/"),col.names=F)
dist_lcss=as.matrix(TSDatabaseDistances(as.matrix(traindata_plane[,2:ncol(traindata_plane)]),distance='lcss',epsilon=0.1))
diag(dist_lcss)=large_number
fwrite(dist_lcss,sprintf('%sPLANE_lcss_raw_epsilon_005.csv',"C:/Users/mirac/HW3/Plane/Distances/Raw/"),col.names=F)
dist_erp=as.matrix(TSDatabaseDistances(as.matrix(traindata_plane[,2:ncol(traindata_plane)]),distance='erp',g=0.5))
diag(dist_erp)=large_number
fwrite(dist_erp,sprintf('%sPLANE_erp_raw_gap_005.csv',"C:/Users/mirac/HW3/Plane/Distances/Raw/"),col.names=F)



nn_classify_cv=function(dist_matrix,train_class,test_indices,k=1){
    
    test_distances_to_train=dist_matrix[test_indices,]
    test_distances_to_train=test_distances_to_train[,-test_indices]
    train_class=train_class[-test_indices]
    ordered_indices=apply(test_distances_to_train,1,order)
    if(k==1){
        nearest_class=as.numeric(trainclass_plane[as.numeric(ordered_indices[1,])])
        nearest_class=data.table(id=test_indices,nearest_class)
    } else {
        nearest_class=apply(ordered_indices[1:k,],2,function(x) {trainclass_plane[x]})
        nearest_class=data.table(id=test_indices,t(nearest_class))
    }
    
    long_nn_class=melt(nearest_class,'id')
    
    class_counts=long_nn_class[,.N,list(id,value)]
    class_counts[,predicted_prob:=N/k]
    wide_class_prob_predictions=dcast(class_counts,id~value,value.var='predicted_prob')
    wide_class_prob_predictions[is.na(wide_class_prob_predictions)]=0
    class_predictions=class_counts[,list(predicted=value[which.max(N)]),by=list(id)]
    
    
    return(list(prediction=class_predictions,prob_estimates=wide_class_prob_predictions))
    
}



set.seed(482)
nof_rep=5
n_fold=10
cv_indices=generateCVRuns(trainclass_plane, ntimes =nof_rep, nfold = n_fold, leaveOneOut = FALSE, stratified = TRUE)
dist_files=list.files("C:/Users/mirac/HW3/Plane/Distances/Raw/", full.names=T)
k_levels=c(1,3,5)
approach_file=list.files("C:/Users/mirac/HW3/Plane/Distances/Raw/")
result=vector('list',length(dist_files)*nof_rep*n_fold*length(k_levels))
iter=1
for(m in 1:length(dist_files)){ 

    dist_mat=as.matrix(fread(dist_files[m],header=FALSE))
    for(i in 1:nof_rep){
        this_fold=cv_indices[[i]]
        for(j in 1:n_fold){
            test_indices=this_fold[[j]]
            for(k in 1:length(k_levels)){
                current_k=k_levels[k]
                current_fold=nn_classify_cv(dist_mat,trainclass_plane,test_indices,k=current_k)
                accuracy=sum(trainclass_plane[test_indices]==current_fold$prediction$predicted)/length(test_indices)
                tmp=data.table(approach=approach_file[m],repid=i,foldid=j,
                               k=current_k,acc=accuracy)
                result[[iter]]=tmp
                iter=iter+1
                
            }
            
        }
        
    }   
    
}
overall_results=rbindlist(result)
summarized_results=overall_results[,list(mean_acc=mean(acc),sdev_acc=sd(acc),result_count=.N),by=list(approach,k)]
summarized_results[order(-mean_acc)]

x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table


approach,k,mean_acc,sdev_acc,result_count
PLANE_erp_raw_gap_005.csv,5,0.72909091,0.13293780,50
PLANE_euc_raw_dist.csv,1,0.71109091,0.12481445,50
PLANE_erp_raw_gap_005.csv,1,0.71018182,0.14287650,50
PLANE_erp_raw_gap_005.csv,3,0.70618182,0.13527007,50
PLANE_euc_raw_dist.csv,3,0.70581818,0.13077297,50
PLANE_dtw_raw_dist.csv,1,0.70563636,0.11911137,50
PLANE_dtw_raw_dist.csv,5,0.69618182,0.13234129,50
PLANE_euc_raw_dist.csv,5,0.69236364,0.12530092,50
PLANE_dtw_raw_dist.csv,3,0.68127273,0.13711494,50
PLANE_lcss_raw_epsilon_005.csv,1,0.11945455,0.06152089,50


In [18]:
dist_euc=as.matrix(dist(paa_rep4[,2:ncol(paa_rep4)]))
diag(dist_euc)=large_number
fwrite(dist_euc,sprintf('%sPLANE_euc_paa4_dist.csv',"C:/Users/mirac/HW3/Plane/Distances/Paa4/"),col.names=F)
dist_dtw=as.matrix(dtwDist(paa_rep4[,2:ncol(paa_rep4)]))
diag(dist_dtw)=large_number
fwrite(dist_dtw,sprintf('%sPLANE_dtw_paa4_dist.csv',"C:/Users/mirac/HW3/Plane/Distances/Paa4/"),col.names=F)
dist_lcss=as.matrix(TSDatabaseDistances(as.matrix(paa_rep4[,2:ncol(paa_rep4)]),distance='lcss',epsilon=0.1))
diag(dist_lcss)=large_number
fwrite(dist_lcss,sprintf('%sPLANE_lcss_paa4_epsilon_005.csv',"C:/Users/mirac/HW3/Plane/Distances/Paa4/"),col.names=F)
dist_erp=as.matrix(TSDatabaseDistances(as.matrix(paa_rep4[,2:ncol(paa_rep4)]),distance='erp',g=0.5))
diag(dist_erp)=large_number
fwrite(dist_erp,sprintf('%sPLANE_erp_paa4_gap_005.csv',"C:/Users/mirac/HW3/Plane/Distances/Paa4/"),col.names=F)

set.seed(482)
nof_rep=5
n_fold=10
cv_indices=generateCVRuns(trainclass_plane, ntimes =nof_rep, nfold = n_fold, leaveOneOut = FALSE, stratified = TRUE)
dist_files=list.files("C:/Users/mirac/HW3/Plane/Distances/Paa4/", full.names=T)
k_levels=c(1,3,5)
approach_file=list.files("C:/Users/mirac/HW3/Plane/Distances/Paa4/")
result=vector('list',length(dist_files)*nof_rep*n_fold*length(k_levels))
iter=1
for(m in 1:length(dist_files)){ 

    dist_mat=as.matrix(fread(dist_files[m],header=FALSE))
    for(i in 1:nof_rep){
        this_fold=cv_indices[[i]]
        for(j in 1:n_fold){
            test_indices=this_fold[[j]]
            for(k in 1:length(k_levels)){
                current_k=k_levels[k]
                current_fold=nn_classify_cv(dist_mat,trainclass_plane,test_indices,k=current_k)
                accuracy=sum(trainclass_plane[test_indices]==current_fold$prediction$predicted)/length(test_indices)
                tmp=data.table(approach=approach_file[m],repid=i,foldid=j,
                               k=current_k,acc=accuracy)
                result[[iter]]=tmp
                iter=iter+1
                
            }
            
        }
        
    }   
    
}
overall_results=rbindlist(result)
summarized_results=overall_results[,list(mean_acc=mean(acc),sdev_acc=sd(acc),result_count=.N),by=list(approach,k)]
summarized_results[order(-mean_acc)]




dist_euc=as.matrix(dist(paa_rep5[,2:ncol(paa_rep5)]))
diag(dist_euc)=large_number
fwrite(dist_euc,sprintf('%sPLANE_euc_paa5_dist.csv',"C:/Users/mirac/HW3/Plane/Distances/Paa5/"),col.names=F)
dist_dtw=as.matrix(dtwDist(paa_rep5[,2:ncol(paa_rep5)]))
diag(dist_dtw)=large_number
fwrite(dist_dtw,sprintf('%sPLANE_dtw_paa5_dist.csv',"C:/Users/mirac/HW3/Plane/Distances/Paa5/"),col.names=F)
dist_lcss=as.matrix(TSDatabaseDistances(as.matrix(paa_rep5[,2:ncol(paa_rep5)]),distance='lcss',epsilon=0.1))
diag(dist_lcss)=large_number
fwrite(dist_lcss,sprintf('%sPLANE_lcss_paa5_epsilon_005.csv',"C:/Users/mirac/HW3/Plane/Distances/Paa5/"),col.names=F)
dist_erp=as.matrix(TSDatabaseDistances(as.matrix(paa_rep5[,2:ncol(paa_rep5)]),distance='erp',g=0.5))
diag(dist_erp)=large_number
fwrite(dist_erp,sprintf('%sPLANE_erp_paa5_gap_005.csv',"C:/Users/mirac/HW3/Plane/Distances/Paa5/"),col.names=F)


set.seed(482)
nof_rep=5
n_fold=10
cv_indices=generateCVRuns(trainclass_plane, ntimes =nof_rep, nfold = n_fold, leaveOneOut = FALSE, stratified = TRUE)
dist_files=list.files("C:/Users/mirac/HW3/Plane/Distances/Paa5/", full.names=T)
k_levels=c(1,3,5)
approach_file=list.files("C:/Users/mirac/HW3/Plane/Distances/Paa5/")
result=vector('list',length(dist_files)*nof_rep*n_fold*length(k_levels))
iter=1
for(m in 1:length(dist_files)){ 

    dist_mat=as.matrix(fread(dist_files[m],header=FALSE))
    for(i in 1:nof_rep){
        this_fold=cv_indices[[i]]
        for(j in 1:n_fold){
            test_indices=this_fold[[j]]
            for(k in 1:length(k_levels)){
                current_k=k_levels[k]
                current_fold=nn_classify_cv(dist_mat,trainclass_plane,test_indices,k=current_k)
                accuracy=sum(trainclass_plane[test_indices]==current_fold$prediction$predicted)/length(test_indices)
                tmp=data.table(approach=approach_file[m],repid=i,foldid=j,
                               k=current_k,acc=accuracy)
                result[[iter]]=tmp
                iter=iter+1
                
            }
            
        }
        
    }   
    
}
overall_results=rbindlist(result)
summarized_results=overall_results[,list(mean_acc=mean(acc),sdev_acc=sd(acc),result_count=.N),by=list(approach,k)]
summarized_results[order(-mean_acc)]

x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table


approach,k,mean_acc,sdev_acc,result_count
PLANE_erp_paa4_gap_005.csv,5,0.79254545,0.12218412,50
PLANE_erp_paa4_gap_005.csv,3,0.76654545,0.11292639,50
PLANE_dtw_paa4_dist.csv,3,0.75781818,0.10237128,50
PLANE_dtw_paa4_dist.csv,5,0.75236364,0.11276377,50
PLANE_euc_paa4_dist.csv,3,0.74872727,0.12912461,50
PLANE_dtw_paa4_dist.csv,1,0.73672727,0.11360423,50
PLANE_euc_paa4_dist.csv,5,0.72909091,0.15164249,50
PLANE_erp_paa4_gap_005.csv,1,0.71381818,0.11284870,50
PLANE_euc_paa4_dist.csv,1,0.70927273,0.12337649,50
PLANE_lcss_paa4_epsilon_005.csv,1,0.06109091,0.07216948,50


x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table


approach,k,mean_acc,sdev_acc,result_count
PLANE_dtw_paa5_dist.csv,3,0.763090909,0.11638291,50
PLANE_dtw_paa5_dist.csv,5,0.757272727,0.11969560,50
PLANE_erp_paa5_gap_005.csv,3,0.757090909,0.11576726,50
PLANE_euc_paa5_dist.csv,3,0.756545455,0.12367415,50
PLANE_euc_paa5_dist.csv,5,0.751818182,0.13287752,50
PLANE_erp_paa5_gap_005.csv,5,0.749636364,0.12367305,50
PLANE_euc_paa5_dist.csv,1,0.711272727,0.12403189,50
PLANE_erp_paa5_gap_005.csv,1,0.678545455,0.11872659,50
PLANE_dtw_paa5_dist.csv,1,0.672909091,0.11694789,50
PLANE_lcss_paa5_epsilon_005.csv,1,0.047818182,0.05558108,50


In [19]:
dist_euc=as.matrix(dist(pla_rep5[,2:ncol(pla_rep5)]))
diag(dist_euc)=large_number
fwrite(dist_euc,sprintf('%sPLANE_euc_pla5_dist.csv',"C:/Users/mirac/HW3/Plane/Distances/Pla5/"),col.names=F)
dist_dtw=as.matrix(dtwDist(pla_rep5[,2:ncol(pla_rep5)]))
diag(dist_dtw)=large_number
fwrite(dist_dtw,sprintf('%sPLANE_dtw_pla5_dist.csv',"C:/Users/mirac/HW3/Plane/Distances/Pla5/"),col.names=F)
dist_lcss=as.matrix(TSDatabaseDistances(as.matrix(pla_rep5[,2:ncol(pla_rep5)]),distance='lcss',epsilon=0.1))
diag(dist_lcss)=large_number
fwrite(dist_lcss,sprintf('%sPLANE_lcss_pla5_epsilon_005.csv',"C:/Users/mirac/HW3/Plane/Distances/Pla5/"),col.names=F)
dist_erp=as.matrix(TSDatabaseDistances(as.matrix(pla_rep5[,2:ncol(pla_rep5)]),distance='erp',g=0.5))
diag(dist_erp)=large_number
fwrite(dist_erp,sprintf('%sPLANE_erp_pla5_gap_005.csv',"C:/Users/mirac/HW3/Plane/Distances/Pla5/"),col.names=F)


set.seed(482)
nof_rep=5
n_fold=10
cv_indices=generateCVRuns(trainclass_plane, ntimes =nof_rep, nfold = n_fold, leaveOneOut = FALSE, stratified = TRUE)
dist_files=list.files("C:/Users/mirac/HW3/Plane/Distances/Pla5/", full.names=T)
k_levels=c(1,3,5)
approach_file=list.files("C:/Users/mirac/HW3/Plane/Distances/Pla5/")
result=vector('list',length(dist_files)*nof_rep*n_fold*length(k_levels))
iter=1
for(m in 1:length(dist_files)){ 

    dist_mat=as.matrix(fread(dist_files[m],header=FALSE))
    for(i in 1:nof_rep){
        this_fold=cv_indices[[i]]
        for(j in 1:n_fold){
            test_indices=this_fold[[j]]
            for(k in 1:length(k_levels)){
                current_k=k_levels[k]
                current_fold=nn_classify_cv(dist_mat,trainclass_plane,test_indices,k=current_k)
                accuracy=sum(trainclass_plane[test_indices]==current_fold$prediction$predicted)/length(test_indices)
                tmp=data.table(approach=approach_file[m],repid=i,foldid=j,
                               k=current_k,acc=accuracy)
                result[[iter]]=tmp
                iter=iter+1
                
            }
            
        }
        
    }   
    
}
overall_results=rbindlist(result)
summarized_results=overall_results[,list(mean_acc=mean(acc),sdev_acc=sd(acc),result_count=.N),by=list(approach,k)]
summarized_results[order(-mean_acc)]




dist_euc=as.matrix(dist(pla_rep10[,2:ncol(pla_rep10)]))
diag(dist_euc)=large_number
fwrite(dist_euc,sprintf('%sPLANE_euc_pla10_dist.csv',"C:/Users/mirac/HW3/Plane/Distances/Pla10/"),col.names=F)
dist_dtw=as.matrix(dtwDist(pla_rep10[,2:ncol(pla_rep10)]))
diag(dist_dtw)=large_number
fwrite(dist_dtw,sprintf('%sPLANE_dtw_pla10_dist.csv',"C:/Users/mirac/HW3/Plane/Distances/Pla10/"),col.names=F)
dist_lcss=as.matrix(TSDatabaseDistances(as.matrix(pla_rep10[,2:ncol(pla_rep10)]),distance='lcss',epsilon=0.1))
diag(dist_lcss)=large_number
fwrite(dist_lcss,sprintf('%sPLANE_lcss_pla10_epsilon_005.csv',"C:/Users/mirac/HW3/Plane/Distances/Pla10/"),col.names=F)
dist_erp=as.matrix(TSDatabaseDistances(as.matrix(pla_rep10[,2:ncol(pla_rep10)]),distance='erp',g=0.5))
diag(dist_erp)=large_number
fwrite(dist_erp,sprintf('%sPLANE_erp_pla10_gap_005.csv',"C:/Users/mirac/HW3/Plane/Distances/Pla10/"),col.names=F)


set.seed(482)
nof_rep=5
n_fold=10
cv_indices=generateCVRuns(trainclass_plane, ntimes =nof_rep, nfold = n_fold, leaveOneOut = FALSE, stratified = TRUE)
dist_files=list.files("C:/Users/mirac/HW3/Plane/Distances/Pla10/", full.names=T)
k_levels=c(1,3,5)
approach_file=list.files("C:/Users/mirac/HW3/Plane/Distances/Pla10/")
result=vector('list',length(dist_files)*nof_rep*n_fold*length(k_levels))
iter=1
for(m in 1:length(dist_files)){ 

    dist_mat=as.matrix(fread(dist_files[m],header=FALSE))
    for(i in 1:nof_rep){
        this_fold=cv_indices[[i]]
        for(j in 1:n_fold){
            test_indices=this_fold[[j]]
            for(k in 1:length(k_levels)){
                current_k=k_levels[k]
                current_fold=nn_classify_cv(dist_mat,trainclass_plane,test_indices,k=current_k)
                accuracy=sum(trainclass_plane[test_indices]==current_fold$prediction$predicted)/length(test_indices)
                tmp=data.table(approach=approach_file[m],repid=i,foldid=j,
                               k=current_k,acc=accuracy)
                result[[iter]]=tmp
                iter=iter+1
                
            }
            
        }
        
    }   
    
}
overall_results=rbindlist(result)
summarized_results=overall_results[,list(mean_acc=mean(acc),sdev_acc=sd(acc),result_count=.N),by=list(approach,k)]
summarized_results[order(-mean_acc)]



x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table


approach,k,mean_acc,sdev_acc,result_count
PLANE_dtw_pla5_dist.csv,1,0.56981818,0.14372159,50
PLANE_dtw_pla5_dist.csv,3,0.55272727,0.14275085,50
PLANE_erp_pla5_gap_005.csv,1,0.46200000,0.13090097,50
PLANE_dtw_pla5_dist.csv,5,0.45636364,0.13994674,50
PLANE_erp_pla5_gap_005.csv,3,0.43981818,0.13496362,50
PLANE_erp_pla5_gap_005.csv,5,0.37254545,0.14518168,50
PLANE_euc_pla5_dist.csv,1,0.36727273,0.13230191,50
PLANE_euc_pla5_dist.csv,3,0.36345455,0.13726345,50
PLANE_euc_pla5_dist.csv,5,0.34418182,0.12572623,50
PLANE_lcss_pla5_epsilon_005.csv,5,0.08018182,0.07715401,50


x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table


approach,k,mean_acc,sdev_acc,result_count
PLANE_euc_pla10_dist.csv,1,0.6109091,0.12776211,50
PLANE_erp_pla10_gap_005.csv,3,0.5818182,0.13114954,50
PLANE_euc_pla10_dist.csv,5,0.5727273,0.15346671,50
PLANE_erp_pla10_gap_005.csv,5,0.5541818,0.17335556,50
PLANE_erp_pla10_gap_005.csv,1,0.5485455,0.15381372,50
PLANE_dtw_pla10_dist.csv,3,0.5390909,0.15321648,50
PLANE_euc_pla10_dist.csv,3,0.5352727,0.11700773,50
PLANE_dtw_pla10_dist.csv,1,0.5260000,0.15901546,50
PLANE_dtw_pla10_dist.csv,5,0.5130909,0.13222744,50
PLANE_lcss_pla10_epsilon_005.csv,3,0.1263636,0.06589549,50


### According to Evaluations Best Parameter Combination for Plane Dataset :
##### Piecewise Aggregate Approximation with segment length 4 and Erp with g=0.5 k=5
##### Obtaining Accuracy Rate for Best Combination
#### It is 99%

In [20]:
testdata_plane=fread("C:/Users/mirac/HW3/Plane/Plane_TEST.txt")
setnames(testdata_plane,'V1','class')
testdata_plane=testdata_plane[order(class)]
testdata_plane[,class:=as.character(class)]
testdata_plane[,id:=1:.N]
longtest_plane=melt(testdata_plane,id.vars=c('id','class'))
longtest_plane[,time:=as.numeric(gsub("\\D", "", variable))-1]
longtest_plane=longtest_plane[,list(id,class,time,value)]
longtest_plane=longtest_plane[order(id,time)]



reprfunc=function(x){
    selected_series=x
    data_plot=longtest_plane[id==x]
    
    
    segment_length=4
    longtest_plane=longtest_plane[order(id,time)]
    data_ts=longtest_plane[id==x]$value
    paa_rep4=repr_paa(data_ts, segment_length, meanC)
    
    dummy_time=c(1:(length(paa_rep4)-1))*segment_length 
    dummy_time=c(dummy_time,nrow(data_plot)) 
    
    dt_paa=data.table(time=dummy_time,paa_rep4=paa_rep4)
    data_plot=merge(data_plot,dt_paa,by='time',all.x=T)
    data_plot[,paa_rep4:=nafill(paa_rep4,'nocb')]
    
   
    return(data_plot)
}


representations=data.table()

for(i in 1:105){
    representations=rbind(representations,reprfunc(i))
}

representations=dcast(representations,id~time,value.var = 'paa_rep4')

testclass_plane=as.matrix(testdata_plane[,1])

dist_erp_test=as.matrix(TSDatabaseDistances(as.matrix(representations[,2:ncol(representations)]),distance='erp',g=0.5))
diag(dist_erp_test)=large_number


neighborhood=apply(dist_erp_test,1,order)
nearest_class=apply(neighborhood[1:5,],2,function(x) {testclass_plane[x]})
nearest_class=data.table(id=1:ncol(neighborhood),t(nearest_class))
long_nn_class=melt(nearest_class,'id')
long_nn_class=long_nn_class[order(id)]
class_counts=long_nn_class[,.N,list(id,value)]
class_predictions=class_counts[,list(predicted=value[which.max(N)]),by=list(id)]
acc=sum(testclass_plane==class_predictions$predicted)/length(testclass_plane)
acc

[1] 0.9904762

## Same Steps for GunPointOldVersusYoung Dataset

In [21]:
traindata_gun=fread("C:/Users/mirac/HW3/GunPointOldVersusYoung/GunPointOldVersusYoung_TRAIN.txt")
 setnames(traindata_gun,'V1','class')
 traindata_gun=traindata_gun[order(class)]
 traindata_gun[,class:=as.character(class)]
 traindata_gun[,id:=1:.N]
 longtrain_gun=melt(traindata_gun,id.vars=c('id','class'))
longtrain_gun[,time:=as.numeric(gsub("\\D", "", variable))-1]
longtrain_gun=longtrain_gun[,list(id,class,time,value)]
longtrain_gun=longtrain_gun[order(id,time)]





reprfunc=function(x){
    selected_series=x
    data_plot=longtrain_gun[id==x]
    
    
    segment_length=5
    longtrain_gun=longtrain_gun[order(id,time)]
    data_ts=longtrain_gun[id==x]$value
    paa_rep5=repr_paa(data_ts, segment_length, meanC)
    
    dummy_time=c(1:(length(paa_rep5)-1))*segment_length 
    dummy_time=c(dummy_time,nrow(data_plot)) 
    
    dt_paa=data.table(time=dummy_time,paa_rep5=paa_rep5)
    data_plot=merge(data_plot,dt_paa,by='time',all.x=T)
    data_plot[,paa_rep5:=nafill(paa_rep5,'nocb')]
    
    
    segment_length=4
    longtrain_gun=longtrain_gun[order(id,time)]
    data_ts=longtrain_gun[id==x]$value
    paa_rep4=repr_paa(data_ts, segment_length, meanC)
    
    dummy_time=c(1:(length(paa_rep4)-1))*segment_length 
    dummy_time=c(dummy_time,nrow(data_plot)) 
    
    dt_paa=data.table(time=dummy_time,paa_rep4=paa_rep4)
    data_plot=merge(data_plot,dt_paa,by='time',all.x=T)
    data_plot[,paa_rep4:=nafill(paa_rep4,'nocb')]
    
    
    n_important = 5
    pla_rep5=repr_pla(data_ts, times = n_important, return = "both")
    setnames(pla_rep5,c('places','points'),c('time','pla_rep5'))
    pla_rep5$time=pla_rep5$time+1
    
    data_plot=merge(data_plot,pla_rep5,by='time',all.x=T)
    data_plot[, pla_rep5 := na.approx(pla_rep5)]
    
    
    n_important = 10
    pla_rep10=repr_pla(data_ts, times = n_important, return = "both")
    setnames(pla_rep10,c('places','points'),c('time','pla_rep10'))
    pla_rep10$time=pla_rep10$time+1
    
    data_plot=merge(data_plot,pla_rep10,by='time',all.x=T)
    data_plot[, pla_rep10 := na.approx(pla_rep10)]
    return(data_plot)
}


representations=data.table()

for(i in 1:136){
    representations=rbind(representations,reprfunc(i))
}


paa_rep5=dcast(representations,id~time,value.var='paa_rep5')
paa_rep4=dcast(representations,id~time,value.var='paa_rep4')
pla_rep5=dcast(representations,id~time,value.var='pla_rep5')
pla_rep10=dcast(representations,id~time,value.var='pla_rep10')






trainclass_gun=as.matrix(traindata_gun[,1])
large_number=10000
dist_euc=as.matrix(dist(traindata_gun[,2:ncol(traindata_gun)]))
diag(dist_euc)=large_number
fwrite(dist_euc,sprintf('%sGUN_euc_raw_dist.csv',"C:/Users/mirac/HW3/GunPointOldVersusYoung/Distances/Raw/"),col.names=F)
dist_dtw=as.matrix(dtwDist(traindata_gun[,2:ncol(traindata_gun)]))
diag(dist_dtw)=large_number
fwrite(dist_dtw,sprintf('%sGUN_dtw_raw_dist.csv',"C:/Users/mirac/HW3/GunPointOldVersusYoung/Distances/Raw/"),col.names=F)
dist_lcss=as.matrix(TSDatabaseDistances(as.matrix(traindata_gun[,2:ncol(traindata_gun)]),distance='lcss',epsilon=0.1))
diag(dist_lcss)=large_number
fwrite(dist_lcss,sprintf('%sGUN_lcss_raw_epsilon_005.csv',"C:/Users/mirac/HW3/GunPointOldVersusYoung/Distances/Raw/"),col.names=F)
dist_erp=as.matrix(TSDatabaseDistances(as.matrix(traindata_gun[,2:ncol(traindata_gun)]),distance='erp',g=0.5))
diag(dist_erp)=large_number
fwrite(dist_erp,sprintf('%sGUN_erp_raw_gap_005.csv',"C:/Users/mirac/HW3/GunPointOldVersusYoung/Distances/Raw/"),col.names=F)



nn_classify_cv=function(dist_matrix,train_class,test_indices,k=1){
    
    test_distances_to_train=dist_matrix[test_indices,]
    test_distances_to_train=test_distances_to_train[,-test_indices]
    train_class=train_class[-test_indices]
    ordered_indices=apply(test_distances_to_train,1,order)
    if(k==1){
        nearest_class=as.numeric(trainclass_gun[as.numeric(ordered_indices[1,])])
        nearest_class=data.table(id=test_indices,nearest_class)
    } else {
        nearest_class=apply(ordered_indices[1:k,],2,function(x) {trainclass_gun[x]})
        nearest_class=data.table(id=test_indices,t(nearest_class))
    }
    
    long_nn_class=melt(nearest_class,'id')
    
    class_counts=long_nn_class[,.N,list(id,value)]
    class_counts[,predicted_prob:=N/k]
    wide_class_prob_predictions=dcast(class_counts,id~value,value.var='predicted_prob')
    wide_class_prob_predictions[is.na(wide_class_prob_predictions)]=0
    class_predictions=class_counts[,list(predicted=value[which.max(N)]),by=list(id)]
    
    
    return(list(prediction=class_predictions,prob_estimates=wide_class_prob_predictions))
    
}



set.seed(482)
nof_rep=5
n_fold=10
cv_indices=generateCVRuns(trainclass_gun, ntimes =nof_rep, nfold = n_fold, leaveOneOut = FALSE, stratified = TRUE)
dist_files=list.files("C:/Users/mirac/HW3/GunPointOldVersusYoung/Distances/Raw/", full.names=T)
k_levels=c(1,3,5)
approach_file=list.files("C:/Users/mirac/HW3/GunPointOldVersusYoung/Distances/Raw/")
result=vector('list',length(dist_files)*nof_rep*n_fold*length(k_levels))
iter=1
for(m in 1:length(dist_files)){ 

    dist_mat=as.matrix(fread(dist_files[m],header=FALSE))
    for(i in 1:nof_rep){
        this_fold=cv_indices[[i]]
        for(j in 1:n_fold){
            test_indices=this_fold[[j]]
            for(k in 1:length(k_levels)){
                current_k=k_levels[k]
                current_fold=nn_classify_cv(dist_mat,trainclass_gun,test_indices,k=current_k)
                accuracy=sum(trainclass_gun[test_indices]==current_fold$prediction$predicted)/length(test_indices)
                tmp=data.table(approach=approach_file[m],repid=i,foldid=j,
                               k=current_k,acc=accuracy)
                result[[iter]]=tmp
                iter=iter+1
                
            }
            
        }
        
    }   
    
}
overall_results=rbindlist(result)
summarized_results=overall_results[,list(mean_acc=mean(acc),sdev_acc=sd(acc),result_count=.N),by=list(approach,k)]
summarized_results[order(-mean_acc)]

x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table


approach,k,mean_acc,sdev_acc,result_count
GUN_erp_raw_gap_005.csv,5,1.0000000,0.00000000,50
GUN_dtw_raw_dist.csv,5,0.9984615,0.01087857,50
GUN_dtw_raw_dist.csv,3,0.9956044,0.01758662,50
GUN_euc_raw_dist.csv,5,0.9897802,0.02560386,50
GUN_erp_raw_gap_005.csv,3,0.9882418,0.02723626,50
GUN_euc_raw_dist.csv,3,0.9809890,0.03242452,50
GUN_erp_raw_gap_005.csv,1,0.9705495,0.03922700,50
GUN_euc_raw_dist.csv,1,0.9646154,0.04522937,50
GUN_dtw_raw_dist.csv,1,0.9632967,0.04737541,50
GUN_lcss_raw_epsilon_005.csv,5,0.5246154,0.06021222,50


In [22]:
dist_euc=as.matrix(dist(paa_rep4[,2:ncol(paa_rep4)]))
diag(dist_euc)=large_number
fwrite(dist_euc,sprintf('%sGUN_euc_paa4_dist.csv',"C:/Users/mirac/HW3/GunPointOldVersusYoung/Distances/Paa4/"),col.names=F)
dist_dtw=as.matrix(dtwDist(paa_rep4[,2:ncol(paa_rep4)]))
diag(dist_dtw)=large_number
fwrite(dist_dtw,sprintf('%sGUN_dtw_paa4_dist.csv',"C:/Users/mirac/HW3/GunPointOldVersusYoung/Distances/Paa4/"),col.names=F)
dist_lcss=as.matrix(TSDatabaseDistances(as.matrix(paa_rep4[,2:ncol(paa_rep4)]),distance='lcss',epsilon=0.1))
diag(dist_lcss)=large_number
fwrite(dist_lcss,sprintf('%sGUN_lcss_paa4_epsilon_005.csv',"C:/Users/mirac/HW3/GunPointOldVersusYoung/Distances/Paa4/"),col.names=F)
dist_erp=as.matrix(TSDatabaseDistances(as.matrix(paa_rep4[,2:ncol(paa_rep4)]),distance='erp',g=0.5))
diag(dist_erp)=large_number
fwrite(dist_erp,sprintf('%sGUN_erp_paa4_gap_005.csv',"C:/Users/mirac/HW3/GunPointOldVersusYoung/Distances/Paa4/"),col.names=F)

set.seed(482)
nof_rep=5
n_fold=10
cv_indices=generateCVRuns(trainclass_gun, ntimes =nof_rep, nfold = n_fold, leaveOneOut = FALSE, stratified = TRUE)
dist_files=list.files("C:/Users/mirac/HW3/GunPointOldVersusYoung/Distances/Paa4/", full.names=T)
k_levels=c(1,3,5)
approach_file=list.files("C:/Users/mirac/HW3/GunPointOldVersusYoung/Distances/Paa4/")
result=vector('list',length(dist_files)*nof_rep*n_fold*length(k_levels))
iter=1
for(m in 1:length(dist_files)){ 

    dist_mat=as.matrix(fread(dist_files[m],header=FALSE))
    for(i in 1:nof_rep){
        this_fold=cv_indices[[i]]
        for(j in 1:n_fold){
            test_indices=this_fold[[j]]
            for(k in 1:length(k_levels)){
                current_k=k_levels[k]
                current_fold=nn_classify_cv(dist_mat,trainclass_gun,test_indices,k=current_k)
                accuracy=sum(trainclass_gun[test_indices]==current_fold$prediction$predicted)/length(test_indices)
                tmp=data.table(approach=approach_file[m],repid=i,foldid=j,
                               k=current_k,acc=accuracy)
                result[[iter]]=tmp
                iter=iter+1
                
            }
            
        }
        
    }   
    
}
overall_results=rbindlist(result)
summarized_results=overall_results[,list(mean_acc=mean(acc),sdev_acc=sd(acc),result_count=.N),by=list(approach,k)]
summarized_results[order(-mean_acc)]




dist_euc=as.matrix(dist(paa_rep5[,2:ncol(paa_rep5)]))
diag(dist_euc)=large_number
fwrite(dist_euc,sprintf('%sGUN_euc_paa5_dist.csv',"C:/Users/mirac/HW3/GunPointOldVersusYoung/Distances/Paa5/"),col.names=F)
dist_dtw=as.matrix(dtwDist(paa_rep5[,2:ncol(paa_rep5)]))
diag(dist_dtw)=large_number
fwrite(dist_dtw,sprintf('%sGUN_dtw_paa5_dist.csv',"C:/Users/mirac/HW3/GunPointOldVersusYoung/Distances/Paa5/"),col.names=F)
dist_lcss=as.matrix(TSDatabaseDistances(as.matrix(paa_rep5[,2:ncol(paa_rep5)]),distance='lcss',epsilon=0.1))
diag(dist_lcss)=large_number
fwrite(dist_lcss,sprintf('%sGUN_lcss_paa5_epsilon_005.csv',"C:/Users/mirac/HW3/GunPointOldVersusYoung/Distances/Paa5/"),col.names=F)
dist_erp=as.matrix(TSDatabaseDistances(as.matrix(paa_rep5[,2:ncol(paa_rep5)]),distance='erp',g=0.5))
diag(dist_erp)=large_number
fwrite(dist_erp,sprintf('%sGUN_erp_paa5_gap_005.csv',"C:/Users/mirac/HW3/GunPointOldVersusYoung/Distances/Paa5/"),col.names=F)


set.seed(482)
nof_rep=5
n_fold=10
cv_indices=generateCVRuns(trainclass_gun, ntimes =nof_rep, nfold = n_fold, leaveOneOut = FALSE, stratified = TRUE)
dist_files=list.files("C:/Users/mirac/HW3/GunPointOldVersusYoung/Distances/Paa5/", full.names=T)
k_levels=c(1,3,5)
approach_file=list.files("C:/Users/mirac/HW3/GunPointOldVersusYoung/Distances/Paa5/")
result=vector('list',length(dist_files)*nof_rep*n_fold*length(k_levels))
iter=1
for(m in 1:length(dist_files)){ 

    dist_mat=as.matrix(fread(dist_files[m],header=FALSE))
    for(i in 1:nof_rep){
        this_fold=cv_indices[[i]]
        for(j in 1:n_fold){
            test_indices=this_fold[[j]]
            for(k in 1:length(k_levels)){
                current_k=k_levels[k]
                current_fold=nn_classify_cv(dist_mat,trainclass_gun,test_indices,k=current_k)
                accuracy=sum(trainclass_gun[test_indices]==current_fold$prediction$predicted)/length(test_indices)
                tmp=data.table(approach=approach_file[m],repid=i,foldid=j,
                               k=current_k,acc=accuracy)
                result[[iter]]=tmp
                iter=iter+1
                
            }
            
        }
        
    }   
    
}
overall_results=rbindlist(result)
summarized_results=overall_results[,list(mean_acc=mean(acc),sdev_acc=sd(acc),result_count=.N),by=list(approach,k)]
summarized_results[order(-mean_acc)]

x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table


approach,k,mean_acc,sdev_acc,result_count
GUN_dtw_paa4_dist.csv,5,1.0000000,0.00000000,50
GUN_erp_paa4_gap_005.csv,5,1.0000000,0.00000000,50
GUN_dtw_paa4_dist.csv,3,0.9912088,0.02406548,50
GUN_euc_paa4_dist.csv,5,0.9897802,0.02560386,50
GUN_erp_paa4_gap_005.csv,3,0.9882418,0.02723626,50
GUN_euc_paa4_dist.csv,3,0.9868132,0.02845301,50
GUN_dtw_paa4_dist.csv,1,0.9821978,0.04143892,50
GUN_erp_paa4_gap_005.csv,1,0.9705495,0.03922700,50
GUN_euc_paa4_dist.csv,1,0.9646154,0.04522937,50
GUN_lcss_paa4_epsilon_005.csv,1,0.4774725,0.02462853,50


x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table


approach,k,mean_acc,sdev_acc,result_count
GUN_erp_paa5_gap_005.csv,5,1.0000000,0.00000000,50
GUN_dtw_paa5_dist.csv,5,0.9971429,0.01413919,50
GUN_euc_paa5_dist.csv,5,0.9940659,0.02034285,50
GUN_erp_paa5_gap_005.csv,3,0.9882418,0.02723626,50
GUN_dtw_paa5_dist.csv,3,0.9882418,0.02723626,50
GUN_euc_paa5_dist.csv,3,0.9868132,0.02845301,50
GUN_erp_paa5_gap_005.csv,1,0.9705495,0.03922700,50
GUN_euc_paa5_dist.csv,1,0.9646154,0.04522937,50
GUN_dtw_paa5_dist.csv,1,0.9518681,0.05805889,50
GUN_lcss_paa5_epsilon_005.csv,1,0.4774725,0.02462853,50


In [23]:
dist_euc=as.matrix(dist(pla_rep5[,2:ncol(pla_rep5)]))
diag(dist_euc)=large_number
fwrite(dist_euc,sprintf('%sGUN_euc_pla5_dist.csv',"C:/Users/mirac/HW3/GunPointOldVersusYoung/Distances/Pla5/"),col.names=F)
dist_dtw=as.matrix(dtwDist(pla_rep5[,2:ncol(pla_rep5)]))
diag(dist_dtw)=large_number
fwrite(dist_dtw,sprintf('%sGUN_dtw_pla5_dist.csv',"C:/Users/mirac/HW3/GunPointOldVersusYoung/Distances/Pla5/"),col.names=F)
dist_lcss=as.matrix(TSDatabaseDistances(as.matrix(pla_rep5[,2:ncol(pla_rep5)]),distance='lcss',epsilon=0.1))
diag(dist_lcss)=large_number
fwrite(dist_lcss,sprintf('%sGUN_lcss_pla5_epsilon_005.csv',"C:/Users/mirac/HW3/GunPointOldVersusYoung/Distances/Pla5/"),col.names=F)
dist_erp=as.matrix(TSDatabaseDistances(as.matrix(pla_rep5[,2:ncol(pla_rep5)]),distance='erp',g=0.5))
diag(dist_erp)=large_number
fwrite(dist_erp,sprintf('%sGUN_erp_pla5_gap_005.csv',"C:/Users/mirac/HW3/GunPointOldVersusYoung/Distances/Pla5/"),col.names=F)


set.seed(482)
nof_rep=5
n_fold=10
cv_indices=generateCVRuns(trainclass_gun, ntimes =nof_rep, nfold = n_fold, leaveOneOut = FALSE, stratified = TRUE)
dist_files=list.files("C:/Users/mirac/HW3/GunPointOldVersusYoung/Distances/Pla5/", full.names=T)
k_levels=c(1,3,5)
approach_file=list.files("C:/Users/mirac/HW3/GunPointOldVersusYoung/Distances/Pla5/")
result=vector('list',length(dist_files)*nof_rep*n_fold*length(k_levels))
iter=1
for(m in 1:length(dist_files)){ 

    dist_mat=as.matrix(fread(dist_files[m],header=FALSE))
    for(i in 1:nof_rep){
        this_fold=cv_indices[[i]]
        for(j in 1:n_fold){
            test_indices=this_fold[[j]]
            for(k in 1:length(k_levels)){
                current_k=k_levels[k]
                current_fold=nn_classify_cv(dist_mat,trainclass_gun,test_indices,k=current_k)
                accuracy=sum(trainclass_gun[test_indices]==current_fold$prediction$predicted)/length(test_indices)
                tmp=data.table(approach=approach_file[m],repid=i,foldid=j,
                               k=current_k,acc=accuracy)
                result[[iter]]=tmp
                iter=iter+1
                
            }
            
        }
        
    }   
    
}
overall_results=rbindlist(result)
summarized_results=overall_results[,list(mean_acc=mean(acc),sdev_acc=sd(acc),result_count=.N),by=list(approach,k)]
summarized_results[order(-mean_acc)]




dist_euc=as.matrix(dist(pla_rep10[,2:ncol(pla_rep10)]))
diag(dist_euc)=large_number
fwrite(dist_euc,sprintf('%sGUN_euc_pla10_dist.csv',"C:/Users/mirac/HW3/GunPointOldVersusYoung/Distances/Pla10/"),col.names=F)
dist_dtw=as.matrix(dtwDist(pla_rep10[,2:ncol(pla_rep10)]))
diag(dist_dtw)=large_number
fwrite(dist_dtw,sprintf('%sGUN_dtw_pla10_dist.csv',"C:/Users/mirac/HW3/GunPointOldVersusYoung/Distances/Pla10/"),col.names=F)
dist_lcss=as.matrix(TSDatabaseDistances(as.matrix(pla_rep10[,2:ncol(pla_rep10)]),distance='lcss',epsilon=0.1))
diag(dist_lcss)=large_number
fwrite(dist_lcss,sprintf('%sGUN_lcss_pla10_epsilon_005.csv',"C:/Users/mirac/HW3/GunPointOldVersusYoung/Distances/Pla10/"),col.names=F)
dist_erp=as.matrix(TSDatabaseDistances(as.matrix(pla_rep10[,2:ncol(pla_rep10)]),distance='erp',g=0.5))
diag(dist_erp)=large_number
fwrite(dist_erp,sprintf('%sGUN_erp_pla10_gap_005.csv',"C:/Users/mirac/HW3/GunPointOldVersusYoung/Distances/Pla10/"),col.names=F)


set.seed(482)
nof_rep=5
n_fold=10
cv_indices=generateCVRuns(trainclass_gun, ntimes =nof_rep, nfold = n_fold, leaveOneOut = FALSE, stratified = TRUE)
dist_files=list.files("C:/Users/mirac/HW3/GunPointOldVersusYoung/Distances/Pla10/", full.names=T)
k_levels=c(1,3,5)
approach_file=list.files("C:/Users/mirac/HW3/GunPointOldVersusYoung/Distances/Pla10/")
result=vector('list',length(dist_files)*nof_rep*n_fold*length(k_levels))
iter=1
for(m in 1:length(dist_files)){ 

    dist_mat=as.matrix(fread(dist_files[m],header=FALSE))
    for(i in 1:nof_rep){
        this_fold=cv_indices[[i]]
        for(j in 1:n_fold){
            test_indices=this_fold[[j]]
            for(k in 1:length(k_levels)){
                current_k=k_levels[k]
                current_fold=nn_classify_cv(dist_mat,trainclass_gun,test_indices,k=current_k)
                accuracy=sum(trainclass_gun[test_indices]==current_fold$prediction$predicted)/length(test_indices)
                tmp=data.table(approach=approach_file[m],repid=i,foldid=j,
                               k=current_k,acc=accuracy)
                result[[iter]]=tmp
                iter=iter+1
                
            }
            
        }
        
    }   
    
}
overall_results=rbindlist(result)
summarized_results=overall_results[,list(mean_acc=mean(acc),sdev_acc=sd(acc),result_count=.N),by=list(approach,k)]
summarized_results[order(-mean_acc)]



x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table


approach,k,mean_acc,sdev_acc,result_count
GUN_erp_pla5_gap_005.csv,5,1.0000000,0.00000000,50
GUN_dtw_pla5_dist.csv,5,0.9985714,0.01010153,50
GUN_erp_pla5_gap_005.csv,3,0.9941758,0.01996284,50
GUN_euc_pla5_dist.csv,5,0.9941758,0.01996284,50
GUN_euc_pla5_dist.csv,3,0.9865934,0.02892888,50
GUN_dtw_pla5_dist.csv,3,0.9824176,0.03163178,50
GUN_euc_pla5_dist.csv,1,0.9717582,0.03923092,50
GUN_dtw_pla5_dist.csv,1,0.9706593,0.04640533,50
GUN_erp_pla5_gap_005.csv,1,0.9627473,0.04035010,50
GUN_lcss_pla5_epsilon_005.csv,5,0.6190110,0.07678367,50


x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table


approach,k,mean_acc,sdev_acc,result_count
GUN_erp_pla10_gap_005.csv,5,1.0000000,0.00000000,50
GUN_dtw_pla10_dist.csv,5,0.9957143,0.01713557,50
GUN_euc_pla10_dist.csv,5,0.9927473,0.02199035,50
GUN_euc_pla10_dist.csv,3,0.9898901,0.02914531,50
GUN_erp_pla10_gap_005.csv,3,0.9882418,0.02723626,50
GUN_dtw_pla10_dist.csv,3,0.9880220,0.02774456,50
GUN_erp_pla10_gap_005.csv,1,0.9720879,0.04142821,50
GUN_euc_pla10_dist.csv,1,0.9705495,0.04459283,50
GUN_dtw_pla10_dist.csv,1,0.9574725,0.05321115,50
GUN_lcss_pla10_epsilon_005.csv,5,0.6069231,0.07869057,50


### According to Evaluations Best Parameter Combination for GunPointOldVersusYoung Dataset:
####  There are multiple bests with 100% rate but I choose Raw with Erp g=0.5 k=5
#### Obtaining Accuracy Rate on Test Data for Best Combination
#### It is 100%

In [24]:
testdata_gun=fread("C:/Users/mirac/HW3/GunPointOldVersusYoung/GunPointOldVersusYoung_TEST.txt")
setnames(testdata_gun,'V1','class')
testdata_gun=testdata_gun[order(class)]
testdata_gun[,class:=as.character(class)]
testdata_gun[,id:=1:.N]
longtest_gun=melt(testdata_gun,id.vars=c('id','class'))
longtest_gun[,time:=as.numeric(gsub("\\D", "", variable))-1]
longtest_gun=longtest_gun[,list(id,class,time,value)]
longtest_gun=longtest_gun[order(id,time)]



testclass_gun=as.matrix(testdata_gun[,1])

dist_erp_test=as.matrix(TSDatabaseDistances(as.matrix(testdata_gun[,2:ncol(testdata_gun)]),distance='erp',g=0.5))
diag(dist_erp_test)=large_number


neighborhood=apply(dist_erp_test,1,order)
nearest_class=apply(neighborhood[1:5,],2,function(x) {testclass_gun[x]})
nearest_class=data.table(id=1:ncol(neighborhood),t(nearest_class))
long_nn_class=melt(nearest_class,'id')
long_nn_class=long_nn_class[order(id)]
class_counts=long_nn_class[,.N,list(id,value)]
class_predictions=class_counts[,list(predicted=value[which.max(N)]),by=list(id)]
acc=sum(testclass_gun==class_predictions$predicted)/length(testclass_gun)
acc


[1] 1

## Same Evaluation Steps for ECG Dataset

In [25]:
traindata_ecg=fread("C:/Users/mirac/HW3/ECG/ECG200_TRAIN.txt")
 setnames(traindata_ecg,'V1','class')
 traindata_ecg=traindata_ecg[order(class)]
 traindata_ecg[,class:=as.character(class)]
 traindata_ecg[,id:=1:.N]
 longtrain_ecg=melt(traindata_ecg,id.vars=c('id','class'))
longtrain_ecg[,time:=as.numeric(gsub("\\D", "", variable))-1]
longtrain_ecg=longtrain_ecg[,list(id,class,time,value)]
longtrain_ecg=longtrain_ecg[order(id,time)]





reprfunc=function(x){
    selected_series=x
    data_plot=longtrain_ecg[id==x]
    
    
    segment_length=5
    longtrain_ecg=longtrain_ecg[order(id,time)]
    data_ts=longtrain_ecg[id==x]$value
    paa_rep5=repr_paa(data_ts, segment_length, meanC)
    
    dummy_time=c(1:(length(paa_rep5)-1))*segment_length 
    dummy_time=c(dummy_time,nrow(data_plot)) 
    
    dt_paa=data.table(time=dummy_time,paa_rep5=paa_rep5)
    data_plot=merge(data_plot,dt_paa,by='time',all.x=T)
    data_plot[,paa_rep5:=nafill(paa_rep5,'nocb')]
    
    
    segment_length=4
    longtrain_ecg=longtrain_ecg[order(id,time)]
    data_ts=longtrain_ecg[id==x]$value
    paa_rep4=repr_paa(data_ts, segment_length, meanC)
    
    dummy_time=c(1:(length(paa_rep4)-1))*segment_length 
    dummy_time=c(dummy_time,nrow(data_plot)) 
    
    dt_paa=data.table(time=dummy_time,paa_rep4=paa_rep4)
    data_plot=merge(data_plot,dt_paa,by='time',all.x=T)
    data_plot[,paa_rep4:=nafill(paa_rep4,'nocb')]
    
    
    n_important = 5
    pla_rep5=repr_pla(data_ts, times = n_important, return = "both")
    setnames(pla_rep5,c('places','points'),c('time','pla_rep5'))
    pla_rep5$time=pla_rep5$time+1
    
    data_plot=merge(data_plot,pla_rep5,by='time',all.x=T)
    data_plot[, pla_rep5 := na.approx(pla_rep5)]
    
    
    n_important = 10
    pla_rep10=repr_pla(data_ts, times = n_important, return = "both")
    setnames(pla_rep10,c('places','points'),c('time','pla_rep10'))
    pla_rep10$time=pla_rep10$time+1
    
    data_plot=merge(data_plot,pla_rep10,by='time',all.x=T)
    data_plot[, pla_rep10 := na.approx(pla_rep10)]
    return(data_plot)
}


representations=data.table()

for(i in 1:100){
    representations=rbind(representations,reprfunc(i))
}


paa_rep5=dcast(representations,id~time,value.var='paa_rep5')
paa_rep4=dcast(representations,id~time,value.var='paa_rep4')
pla_rep5=dcast(representations,id~time,value.var='pla_rep5')
pla_rep10=dcast(representations,id~time,value.var='pla_rep10')






trainclass_ecg=as.matrix(traindata_ecg[,1])
large_number=10000
dist_euc=as.matrix(dist(traindata_ecg[,2:ncol(traindata_ecg)]))
diag(dist_euc)=large_number
fwrite(dist_euc,sprintf('%sECG_euc_raw_dist.csv',"C:/Users/mirac/HW3/ECG/Distances/Raw/"),col.names=F)
dist_dtw=as.matrix(dtwDist(traindata_ecg[,2:ncol(traindata_ecg)]))
diag(dist_dtw)=large_number
fwrite(dist_dtw,sprintf('%sECG_dtw_raw_dist.csv',"C:/Users/mirac/HW3/ECG/Distances/Raw/"),col.names=F)
dist_lcss=as.matrix(TSDatabaseDistances(as.matrix(traindata_ecg[,2:ncol(traindata_ecg)]),distance='lcss',epsilon=0.1))
diag(dist_lcss)=large_number
fwrite(dist_lcss,sprintf('%sECG_lcss_raw_epsilon_005.csv',"C:/Users/mirac/HW3/ECG/Distances/Raw/"),col.names=F)
dist_erp=as.matrix(TSDatabaseDistances(as.matrix(traindata_ecg[,2:ncol(traindata_ecg)]),distance='erp',g=0.5))
diag(dist_erp)=large_number
fwrite(dist_erp,sprintf('%sECG_erp_raw_gap_005.csv',"C:/Users/mirac/HW3/ECG/Distances/Raw/"),col.names=F)



nn_classify_cv=function(dist_matrix,train_class,test_indices,k=1){
    
    test_distances_to_train=dist_matrix[test_indices,]
    test_distances_to_train=test_distances_to_train[,-test_indices]
    train_class=train_class[-test_indices]
    ordered_indices=apply(test_distances_to_train,1,order)
    if(k==1){
        nearest_class=as.numeric(trainclass_ecg[as.numeric(ordered_indices[1,])])
        nearest_class=data.table(id=test_indices,nearest_class)
    } else {
        nearest_class=apply(ordered_indices[1:k,],2,function(x) {trainclass_ecg[x]})
        nearest_class=data.table(id=test_indices,t(nearest_class))
    }
    
    long_nn_class=melt(nearest_class,'id')
    
    class_counts=long_nn_class[,.N,list(id,value)]
    class_counts[,predicted_prob:=N/k]
    wide_class_prob_predictions=dcast(class_counts,id~value,value.var='predicted_prob')
    wide_class_prob_predictions[is.na(wide_class_prob_predictions)]=0
    class_predictions=class_counts[,list(predicted=value[which.max(N)]),by=list(id)]
    
    
    return(list(prediction=class_predictions,prob_estimates=wide_class_prob_predictions))
    
}



set.seed(482)
nof_rep=5
n_fold=10
cv_indices=generateCVRuns(trainclass_ecg, ntimes =nof_rep, nfold = n_fold, leaveOneOut = FALSE, stratified = TRUE)
dist_files=list.files("C:/Users/mirac/HW3/ECG/Distances/Raw/", full.names=T)
k_levels=c(1,3,5)
approach_file=list.files("C:/Users/mirac/HW3/ECG/Distances/Raw/")
result=vector('list',length(dist_files)*nof_rep*n_fold*length(k_levels))
iter=1
for(m in 1:length(dist_files)){ 

    dist_mat=as.matrix(fread(dist_files[m],header=FALSE))
    for(i in 1:nof_rep){
        this_fold=cv_indices[[i]]
        for(j in 1:n_fold){
            test_indices=this_fold[[j]]
            for(k in 1:length(k_levels)){
                current_k=k_levels[k]
                current_fold=nn_classify_cv(dist_mat,trainclass_ecg,test_indices,k=current_k)
                accuracy=sum(trainclass_ecg[test_indices]==current_fold$prediction$predicted)/length(test_indices)
                tmp=data.table(approach=approach_file[m],repid=i,foldid=j,
                               k=current_k,acc=accuracy)
                result[[iter]]=tmp
                iter=iter+1
                
            }
            
        }
        
    }   
    
}
overall_results=rbindlist(result)
summarized_results=overall_results[,list(mean_acc=mean(acc),sdev_acc=sd(acc),result_count=.N),by=list(approach,k)]
summarized_results[order(-mean_acc)]


x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table


approach,k,mean_acc,sdev_acc,result_count
ECG_dtw_raw_dist.csv,1,0.960,0.07284314,50
ECG_dtw_raw_dist.csv,3,0.956,0.08121526,50
ECG_erp_raw_gap_005.csv,5,0.956,0.06749150,50
ECG_euc_raw_dist.csv,1,0.950,0.07071068,50
ECG_dtw_raw_dist.csv,5,0.948,0.07886956,50
ECG_erp_raw_gap_005.csv,3,0.942,0.07024738,50
ECG_euc_raw_dist.csv,3,0.942,0.07583548,50
ECG_euc_raw_dist.csv,5,0.942,0.07309485,50
ECG_erp_raw_gap_005.csv,1,0.940,0.07284314,50
ECG_lcss_raw_epsilon_005.csv,5,0.320,0.04517540,50


In [26]:
dist_euc=as.matrix(dist(paa_rep4[,2:ncol(paa_rep4)]))
diag(dist_euc)=large_number
fwrite(dist_euc,sprintf('%sECG_euc_paa4_dist.csv',"C:/Users/mirac/HW3/ECG/Distances/Paa4/"),col.names=F)
dist_dtw=as.matrix(dtwDist(paa_rep4[,2:ncol(paa_rep4)]))
diag(dist_dtw)=large_number
fwrite(dist_dtw,sprintf('%sECG_dtw_paa4_dist.csv',"C:/Users/mirac/HW3/ECG/Distances/Paa4/"),col.names=F)
dist_lcss=as.matrix(TSDatabaseDistances(as.matrix(paa_rep4[,2:ncol(paa_rep4)]),distance='lcss',epsilon=0.1))
diag(dist_lcss)=large_number
fwrite(dist_lcss,sprintf('%sECG_lcss_paa4_epsilon_005.csv',"C:/Users/mirac/HW3/ECG/Distances/Paa4/"),col.names=F)
dist_erp=as.matrix(TSDatabaseDistances(as.matrix(paa_rep4[,2:ncol(paa_rep4)]),distance='erp',g=0.5))
diag(dist_erp)=large_number
fwrite(dist_erp,sprintf('%sECG_erp_paa4_gap_005.csv',"C:/Users/mirac/HW3/ECG/Distances/Paa4/"),col.names=F)

set.seed(482)
nof_rep=5
n_fold=10
cv_indices=generateCVRuns(trainclass_ecg, ntimes =nof_rep, nfold = n_fold, leaveOneOut = FALSE, stratified = TRUE)
dist_files=list.files("C:/Users/mirac/HW3/ECG/Distances/Paa4/", full.names=T)
k_levels=c(1,3,5)
approach_file=list.files("C:/Users/mirac/HW3/ECG/Distances/Paa4/")
result=vector('list',length(dist_files)*nof_rep*n_fold*length(k_levels))
iter=1
for(m in 1:length(dist_files)){ 

    dist_mat=as.matrix(fread(dist_files[m],header=FALSE))
    for(i in 1:nof_rep){
        this_fold=cv_indices[[i]]
        for(j in 1:n_fold){
            test_indices=this_fold[[j]]
            for(k in 1:length(k_levels)){
                current_k=k_levels[k]
                current_fold=nn_classify_cv(dist_mat,trainclass_ecg,test_indices,k=current_k)
                accuracy=sum(trainclass_ecg[test_indices]==current_fold$prediction$predicted)/length(test_indices)
                tmp=data.table(approach=approach_file[m],repid=i,foldid=j,
                               k=current_k,acc=accuracy)
                result[[iter]]=tmp
                iter=iter+1
                
            }
            
        }
        
    }   
    
}
overall_results=rbindlist(result)
summarized_results=overall_results[,list(mean_acc=mean(acc),sdev_acc=sd(acc),result_count=.N),by=list(approach,k)]
summarized_results[order(-mean_acc)]




dist_euc=as.matrix(dist(paa_rep5[,2:ncol(paa_rep5)]))
diag(dist_euc)=large_number
fwrite(dist_euc,sprintf('%sECG_euc_paa5_dist.csv',"C:/Users/mirac/HW3/ECG/Distances/Paa5/"),col.names=F)
dist_dtw=as.matrix(dtwDist(paa_rep5[,2:ncol(paa_rep5)]))
diag(dist_dtw)=large_number
fwrite(dist_dtw,sprintf('%sECG_dtw_paa5_dist.csv',"C:/Users/mirac/HW3/ECG/Distances/Paa5/"),col.names=F)
dist_lcss=as.matrix(TSDatabaseDistances(as.matrix(paa_rep5[,2:ncol(paa_rep5)]),distance='lcss',epsilon=0.1))
diag(dist_lcss)=large_number
fwrite(dist_lcss,sprintf('%sECG_lcss_paa5_epsilon_005.csv',"C:/Users/mirac/HW3/ECG/Distances/Paa5/"),col.names=F)
dist_erp=as.matrix(TSDatabaseDistances(as.matrix(paa_rep5[,2:ncol(paa_rep5)]),distance='erp',g=0.5))
diag(dist_erp)=large_number
fwrite(dist_erp,sprintf('%sECG_erp_paa5_gap_005.csv',"C:/Users/mirac/HW3/ECG/Distances/Paa5/"),col.names=F)


set.seed(482)
nof_rep=5
n_fold=10
cv_indices=generateCVRuns(trainclass_ecg, ntimes =nof_rep, nfold = n_fold, leaveOneOut = FALSE, stratified = TRUE)
dist_files=list.files("C:/Users/mirac/HW3/ECG/Distances/Paa5/", full.names=T)
k_levels=c(1,3,5)
approach_file=list.files("C:/Users/mirac/HW3/ECG/Distances/Paa5/")
result=vector('list',length(dist_files)*nof_rep*n_fold*length(k_levels))
iter=1
for(m in 1:length(dist_files)){ 

    dist_mat=as.matrix(fread(dist_files[m],header=FALSE))
    for(i in 1:nof_rep){
        this_fold=cv_indices[[i]]
        for(j in 1:n_fold){
            test_indices=this_fold[[j]]
            for(k in 1:length(k_levels)){
                current_k=k_levels[k]
                current_fold=nn_classify_cv(dist_mat,trainclass_ecg,test_indices,k=current_k)
                accuracy=sum(trainclass_ecg[test_indices]==current_fold$prediction$predicted)/length(test_indices)
                tmp=data.table(approach=approach_file[m],repid=i,foldid=j,
                               k=current_k,acc=accuracy)
                result[[iter]]=tmp
                iter=iter+1
                
            }
            
        }
        
    }   
    
}
overall_results=rbindlist(result)
summarized_results=overall_results[,list(mean_acc=mean(acc),sdev_acc=sd(acc),result_count=.N),by=list(approach,k)]
summarized_results[order(-mean_acc)]

x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table


approach,k,mean_acc,sdev_acc,result_count
ECG_euc_paa4_dist.csv,5,0.846,0.09733175,50
ECG_euc_paa4_dist.csv,3,0.844,0.09930370,50
ECG_euc_paa4_dist.csv,1,0.790,0.12330483,50
ECG_erp_paa4_gap_005.csv,5,0.770,0.13286482,50
ECG_erp_paa4_gap_005.csv,1,0.764,0.12414606,50
ECG_dtw_paa4_dist.csv,5,0.762,0.10669430,50
ECG_dtw_paa4_dist.csv,3,0.760,0.13997084,50
ECG_erp_paa4_gap_005.csv,3,0.732,0.14061091,50
ECG_dtw_paa4_dist.csv,1,0.722,0.13596218,50
ECG_lcss_paa4_epsilon_005.csv,1,0.376,0.09160697,50


x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table


approach,k,mean_acc,sdev_acc,result_count
ECG_euc_paa5_dist.csv,3,0.868,0.09987748,50
ECG_euc_paa5_dist.csv,5,0.832,0.09987748,50
ECG_erp_paa5_gap_005.csv,1,0.830,0.08630747,50
ECG_euc_paa5_dist.csv,1,0.806,0.12356936,50
ECG_erp_paa5_gap_005.csv,3,0.794,0.11851117,50
ECG_erp_paa5_gap_005.csv,5,0.780,0.12616801,50
ECG_dtw_paa5_dist.csv,3,0.762,0.14126253,50
ECG_dtw_paa5_dist.csv,1,0.738,0.12270755,50
ECG_dtw_paa5_dist.csv,5,0.732,0.12687773,50
ECG_lcss_paa5_epsilon_005.csv,1,0.412,0.13797367,50


In [27]:
dist_euc=as.matrix(dist(pla_rep5[,2:ncol(pla_rep5)]))
diag(dist_euc)=large_number
fwrite(dist_euc,sprintf('%sECG_euc_pla5_dist.csv',"C:/Users/mirac/HW3/ECG/Distances/Pla5/"),col.names=F)
dist_dtw=as.matrix(dtwDist(pla_rep5[,2:ncol(pla_rep5)]))
diag(dist_dtw)=large_number
fwrite(dist_dtw,sprintf('%sECG_dtw_pla5_dist.csv',"C:/Users/mirac/HW3/ECG/Distances/Pla5/"),col.names=F)
dist_lcss=as.matrix(TSDatabaseDistances(as.matrix(pla_rep5[,2:ncol(pla_rep5)]),distance='lcss',epsilon=0.1))
diag(dist_lcss)=large_number
fwrite(dist_lcss,sprintf('%sECG_lcss_pla5_epsilon_005.csv',"C:/Users/mirac/HW3/ECG/Distances/Pla5/"),col.names=F)
dist_erp=as.matrix(TSDatabaseDistances(as.matrix(pla_rep5[,2:ncol(pla_rep5)]),distance='erp',g=0.5))
diag(dist_erp)=large_number
fwrite(dist_erp,sprintf('%sECG_erp_pla5_gap_005.csv',"C:/Users/mirac/HW3/ECG/Distances/Pla5/"),col.names=F)


set.seed(482)
nof_rep=5
n_fold=10
cv_indices=generateCVRuns(trainclass_ecg, ntimes =nof_rep, nfold = n_fold, leaveOneOut = FALSE, stratified = TRUE)
dist_files=list.files("C:/Users/mirac/HW3/ECG/Distances/Pla5/", full.names=T)
k_levels=c(1,3,5)
approach_file=list.files("C:/Users/mirac/HW3/ECG/Distances/Pla5/")
result=vector('list',length(dist_files)*nof_rep*n_fold*length(k_levels))
iter=1
for(m in 1:length(dist_files)){ 

    dist_mat=as.matrix(fread(dist_files[m],header=FALSE))
    for(i in 1:nof_rep){
        this_fold=cv_indices[[i]]
        for(j in 1:n_fold){
            test_indices=this_fold[[j]]
            for(k in 1:length(k_levels)){
                current_k=k_levels[k]
                current_fold=nn_classify_cv(dist_mat,trainclass_ecg,test_indices,k=current_k)
                accuracy=sum(trainclass_ecg[test_indices]==current_fold$prediction$predicted)/length(test_indices)
                tmp=data.table(approach=approach_file[m],repid=i,foldid=j,
                               k=current_k,acc=accuracy)
                result[[iter]]=tmp
                iter=iter+1
                
            }
            
        }
        
    }   
    
}
overall_results=rbindlist(result)
summarized_results=overall_results[,list(mean_acc=mean(acc),sdev_acc=sd(acc),result_count=.N),by=list(approach,k)]
summarized_results[order(-mean_acc)]




dist_euc=as.matrix(dist(pla_rep10[,2:ncol(pla_rep10)]))
diag(dist_euc)=large_number
fwrite(dist_euc,sprintf('%sECG_euc_pla10_dist.csv',"C:/Users/mirac/HW3/ECG/Distances/Pla10/"),col.names=F)
dist_dtw=as.matrix(dtwDist(pla_rep10[,2:ncol(pla_rep10)]))
diag(dist_dtw)=large_number
fwrite(dist_dtw,sprintf('%sECG_dtw_pla10_dist.csv',"C:/Users/mirac/HW3/ECG/Distances/Pla10/"),col.names=F)
dist_lcss=as.matrix(TSDatabaseDistances(as.matrix(pla_rep10[,2:ncol(pla_rep10)]),distance='lcss',epsilon=0.1))
diag(dist_lcss)=large_number
fwrite(dist_lcss,sprintf('%sECG_lcss_pla10_epsilon_005.csv',"C:/Users/mirac/HW3/ECG/Distances/Pla10/"),col.names=F)
dist_erp=as.matrix(TSDatabaseDistances(as.matrix(pla_rep10[,2:ncol(pla_rep10)]),distance='erp',g=0.5))
diag(dist_erp)=large_number
fwrite(dist_erp,sprintf('%sECG_erp_pla10_gap_005.csv',"C:/Users/mirac/HW3/ECG/Distances/Pla10/"),col.names=F)


set.seed(482)
nof_rep=5
n_fold=10
cv_indices=generateCVRuns(trainclass_ecg, ntimes =nof_rep, nfold = n_fold, leaveOneOut = FALSE, stratified = TRUE)
dist_files=list.files("C:/Users/mirac/HW3/ECG/Distances/Pla10/", full.names=T)
k_levels=c(1,3,5)
approach_file=list.files("C:/Users/mirac/HW3/ECG/Distances/Pla10/")
result=vector('list',length(dist_files)*nof_rep*n_fold*length(k_levels))
iter=1
for(m in 1:length(dist_files)){ 

    dist_mat=as.matrix(fread(dist_files[m],header=FALSE))
    for(i in 1:nof_rep){
        this_fold=cv_indices[[i]]
        for(j in 1:n_fold){
            test_indices=this_fold[[j]]
            for(k in 1:length(k_levels)){
                current_k=k_levels[k]
                current_fold=nn_classify_cv(dist_mat,trainclass_ecg,test_indices,k=current_k)
                accuracy=sum(trainclass_ecg[test_indices]==current_fold$prediction$predicted)/length(test_indices)
                tmp=data.table(approach=approach_file[m],repid=i,foldid=j,
                               k=current_k,acc=accuracy)
                result[[iter]]=tmp
                iter=iter+1
                
            }
            
        }
        
    }   
    
}
overall_results=rbindlist(result)
summarized_results=overall_results[,list(mean_acc=mean(acc),sdev_acc=sd(acc),result_count=.N),by=list(approach,k)]
summarized_results[order(-mean_acc)]



x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table


approach,k,mean_acc,sdev_acc,result_count
ECG_euc_pla5_dist.csv,1,0.782,0.1350586,50
ECG_euc_pla5_dist.csv,5,0.742,0.1472053,50
ECG_dtw_pla5_dist.csv,3,0.740,0.1106567,50
ECG_euc_pla5_dist.csv,3,0.732,0.1463013,50
ECG_dtw_pla5_dist.csv,5,0.728,0.1069808,50
ECG_erp_pla5_gap_005.csv,1,0.716,0.1314907,50
ECG_erp_pla5_gap_005.csv,3,0.700,0.1538619,50
ECG_erp_pla5_gap_005.csv,5,0.692,0.1337680,50
ECG_dtw_pla5_dist.csv,1,0.644,0.1592841,50
ECG_lcss_pla5_epsilon_005.csv,5,0.402,0.1634950,50


x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table


approach,k,mean_acc,sdev_acc,result_count
ECG_euc_pla10_dist.csv,5,0.822,0.1165665,50
ECG_euc_pla10_dist.csv,3,0.794,0.1331104,50
ECG_euc_pla10_dist.csv,1,0.770,0.1343921,50
ECG_dtw_pla10_dist.csv,5,0.760,0.1049781,50
ECG_dtw_pla10_dist.csv,3,0.756,0.1214874,50
ECG_erp_pla10_gap_005.csv,5,0.734,0.1287459,50
ECG_erp_pla10_gap_005.csv,3,0.708,0.1226244,50
ECG_erp_pla10_gap_005.csv,1,0.688,0.1379737,50
ECG_dtw_pla10_dist.csv,1,0.674,0.1549325,50
ECG_lcss_pla10_epsilon_005.csv,1,0.336,0.1224911,50


### According to Evaluations Best Parameter Combination for ECG Dataset is
#### Raw Dtw k=1
#### Obtaining Accuracy Rate for Best Combination
#### It is 98%

In [28]:
testdata_ecg=fread("C:/Users/mirac/HW3/ECG/ECG200_TEST.txt")
setnames(testdata_ecg,'V1','class')
testdata_ecg=testdata_ecg[order(class)]
testdata_ecg[,class:=as.character(class)]
testdata_ecg[,id:=1:.N]
longtest_ecg=melt(testdata_ecg,id.vars=c('id','class'))
longtest_ecg[,time:=as.numeric(gsub("\\D", "", variable))-1]
longtest_ecg=longtest_ecg[,list(id,class,time,value)]
longtest_ecg=longtest_ecg[order(id,time)]



testclass_ecg=as.matrix(testdata_ecg[,1])

dist_dtw=as.matrix(dtwDist(testdata_ecg[,2:ncol(traindata_ecg)]))
diag(dist_dtw)=large_number


neighborhood=apply(dist_dtw,1,order)
predicted=testclass_ecg[neighborhood[1,]]
acc=sum(testclass_ecg==predicted)/length(predicted)
print(acc)

[1] 0.98


## As a summary 
### I evaluate euclidean distance, dtw, lcss epsilon=0.1, erp g=0.5 with k=1, 3, 5
### Representation Alternatives= Raw data / Paa segment length 4 and 5 / Pla important points 5 and 10
##### For Trace Dataset Best Combnation is Raw, Erp g=0.5 with k=5 and Testdata Accuracy Rate= 95%
##### For SmoothSubspace Dataset Best Combination is Pla 10 important points, Erp g=0.5 with k=5 and Testdata Accuracy Rate=97%
##### For Plane Dataset Best Combination is Paa segment length=4,Erp g=0.5 with k=5 and Testdata Accuracy Rate=99%
##### For GunPointOldVersusYoung Dataset Best Combination is Raw,Erp g=0.5 with k=5 and Testdata Accuracy Rate=100%
##### For ECG Dataset Best Combination is Raw, Dtw, k=1 and Testdata Accuracy Rate= 98%